In [9]:
import os
import torch 
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
 
from torchvision import datasets
from torchvision.utils import save_image
import numpy as np
from torch.utils.data import Dataset, DataLoader
import pickle
import numpy as np
import pandas as pd
from ast import literal_eval
import glob

from PIL import Image, ImageDraw

import time

import cv2


ModuleNotFoundError: No module named 'cv2'

In [79]:


class myDataset(torch.utils.data.Dataset):
    def __init__(self, args, dtype,fname):
        
        self.args = args
        self.dtype = dtype
        print("Loading",self.dtype)
        sequence_centric = pd.read_csv(fname+self.dtype+".csv")
        df = sequence_centric.copy()      
        for v in list(df.columns.values):
            print(v+' loaded')
            try:
                df.loc[:,v] = df.loc[:, v].apply(lambda x: literal_eval(x))
            except:
                continue
        sequence_centric[df.columns] = df[df.columns]
        self.data = sequence_centric.copy().reset_index(drop=True)
    
        print('*'*30)
        

    def __len__(self):
        return len(self.data)
    
    
    def __getitem__(self, index):

        seq = self.data.iloc[index]
        outputs = []

        obs = torch.tensor([seq.Pose[i] for i in range(0,self.args.input,self.args.skip)])
        obs_speed = (obs[1:] - obs[:-1])
        outputs.append(obs_speed)
        true = torch.tensor([seq.Future_Pose[i] for i in range(0,self.args.output,self.args.skip)])
        true_speed = torch.cat(((true[0]-obs[-1]).unsqueeze(0), true[1:]-true[:-1]))
        outputs.append(true_speed)
        outputs.append(obs)
        outputs.append(true)
        
        return tuple(outputs)    
    
    
def data_loader(args,data,fname):
    dataset = myDataset(args,data,fname)
    dataloader = torch.utils.data.DataLoader(
        dataset, batch_size=args.batch_size, shuffle=args.loader_shuffle,
        pin_memory=args.pin_memory)

    return dataloader


In [80]:
def ADE_c(pred, true):
    b,n,p=pred.size()[0],pred.size()[1],pred.size()[2]
#     print(b,n,p)
    pred = torch.reshape(pred, (b,n,int(p/2),2))
    true = torch.reshape(true, (b,n,int(p/2),2))
    
    displacement=torch.sqrt((pred[:,:,:,0]-true[:,:,:,0])**2+(pred[:,:,:,1]-true[:,:,:,1])**2)
    ade = torch.mean(torch.mean(displacement,dim=1))

    return ade


def FDE_c(pred, true):
    b,n,p=pred.size()[0],pred.size()[1],pred.size()[2]
#     print(b,n,p)
    pred = torch.reshape(pred, (b,n,int(p/2),2))
    true = torch.reshape(true, (b,n,int(p/2),2))
    
    displacement=torch.sqrt((pred[:,-1,:,0]-true[:,-1,:,0])**2+(pred[:,-1,:,1]-true[:,-1,:,1])**2)

    fde = torch.mean(torch.mean(displacement,dim=1))
    
    return fde

def speed2pos(preds, obs_p):
    pred_pos = torch.zeros(preds.shape[0], preds.shape[1], preds.shape[2]).to('cuda')
    current = obs_p[:,-1,:]
    
    for i in range(preds.shape[1]):
        pred_pos[:,i,:] = current + preds[:,i,:]
        current = pred_pos[:,i,:]
        
    for i in range(preds.shape[2]):
        pred_pos[:,:,i] = torch.min(pred_pos[:,:,i], 1920*torch.ones(pred_pos.shape[0], pred_pos.shape[1], device='cuda'))
        pred_pos[:,:,i] = torch.max(pred_pos[:,:,i], torch.zeros(pred_pos.shape[0], pred_pos.shape[1], device='cuda'))
        
    return pred_pos

In [81]:
class PoseDataset(Dataset):

    def __init__(self,keypoints, transform=None):
        self.data=keypoints
      
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        image = self.data[idx]

        sample = {'keypoints': torch.from_numpy(image)}

        return sample


In [5]:
def train(net, trainloader,valloader, NUM_EPOCHS, scheduler):
    train_loss = []
    val_losses = []
    max_val_loss=10000
    for epoch in range(NUM_EPOCHS):
        counter=0
        ade_train=0
        ade_val=0
        
        net.train()
        running_loss = 0.0
        running_val_loss = 0.0
        for data in trainloader:
            counter+=1
            kp = data['keypoints']
            kp = kp.to(device)
            
            optimizer.zero_grad()
            outputs = net(kp.float())
            
            loss = criterion(outputs, kp.float())
            
            ade_train += float(ADE_c(outputs.unsqueeze(0), kp.unsqueeze(0)))
            
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        loss = running_loss / len(trainloader)
        train_loss.append(loss)
        
        ade_train/=counter
        
        net.eval()
        counter=0
        with torch.no_grad():
          for data in valloader:
              counter+=1
              kp = data['keypoints']
              kp = kp.to(device)
              outputs = net(kp.float())
              
              ade_val += float(ADE_c(outputs.unsqueeze(0), kp.unsqueeze(0)))
              
              val_loss = criterion(outputs, kp.float())
              running_val_loss += val_loss.item()
          
        val_loss = running_val_loss / len(valloader)
        val_losses.append(val_loss)
        ade_val/=counter
         

        scheduler.step(val_loss) 

#         if(epoch%20==19):
        print('Epoch {} of {}, Train Loss: {:.3f}, Val Loss: {:.3f}, ADE_train : {:.3f},  ADE_val: {:.3f}'.format(
            epoch+1, NUM_EPOCHS, loss, val_loss,ade_train, ade_val))
              
        if val_loss<max_val_loss:
          max_val_loss=val_loss
          torch.save(net.state_dict(), "model_clean_auto.weight")

    return train_loss,val_losses

        

In [5]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        # encoder
        self.drop =  nn.Dropout(p=0.3)
        self.enc1= nn.Linear(in_features=34*16, out_features=300)
        self.enc2 = nn.Linear(in_features=300, out_features=300)
#         self.enc3 = nn.Linear(in_features=150, out_features=75)
#         self.enc4 = nn.Linear(in_features=300, out_features=10)
        # decoder 
#         self.dec1 = nn.Linear(in_features=10, out_features=300)
#         self.dec2 = nn.Linear(in_features=75, out_features=150)
#         self.dec3 = nn.Linear(in_features=300, out_features=300)
        self.dec4 = nn.Linear(in_features=300, out_features=34*16)
        
    def forward(self, x):
                
        x = F.relu(self.drop(x))
        x = F.relu(self.enc1(x))
        x = F.relu(self.enc2(x))
#         x = F.relu(self.enc3(x))
#         x = F.relu(self.enc4(x))
#         x = F.relu(self.dec1(x))
#         x = F.relu(self.dec2(x))
#         x = F.relu(self.dec3(x))
        x = F.relu(self.dec4(x))
        return x
net = Autoencoder()
print(net)

Autoencoder(
  (drop): Dropout(p=0.3, inplace=False)
  (enc1): Linear(in_features=544, out_features=300, bias=True)
  (enc2): Linear(in_features=300, out_features=300, bias=True)
  (dec4): Linear(in_features=300, out_features=544, bias=True)
)


In [191]:
kpa=[]
keypoints=pickle.load(open("keypoints_openpifpaf.pickle","rb"))
kpd={}
for i in keypoints:
	kpd[i]=np.array(keypoints[i])
print(len(kpd))
kpa=[]
for key in list(kpd.keys()):
  x=key.replace(".png.predictions.json","").split("_")
  vid_num=int(x[2])
  frame_num=int(x[3])
  ped_num=x[4]
  if(len(ped_num)==11):
    ped_num=int(x[4][10])
  else:
    ped_num=0
    
#   if np.count_nonzero(kpd[key]<=0)<=9:
#       kpa.append([vid_num,frame_num,ped_num,kpd[key]])
  vec=kpd[key][2::3]
  if (vec > 0.25).all():
      kpa.append([vid_num,frame_num,ped_num,kpd[key]])


keypoints_array_sorted=sorted(kpa,key=lambda e:(e[0],e[2],e[1]))
print(len(keypoints_array_sorted))
thresarr=[]
for i in range(len(keypoints_array_sorted)):
  vec=keypoints_array_sorted[i][3][2::3]
  if (vec > 0.25).all():
      thresarr.append(keypoints_array_sorted[i][3])
keypoints=np.array(thresarr)
kp_train=np.delete(keypoints, list(range(2, keypoints.shape[1], 3)), axis=1)
print(kp_train.shape)

95233
4711
(4711, 34)


In [142]:
import itertools

key_f = lambda e: (e[0],e[2])
kp_seq=[]
for key, group in itertools.groupby(keypoints_array_sorted, key_f):
    lg=list(group)
    # print(len(lg))
    kp_seq.append(lg)

print(len(keypoints_array_sorted))
gps=[]
for seq in kp_seq:
  l=[]
  for i in range(len(seq)):
#     print(seq[i][3].shape)
    vec=seq[i][3][2::3]
    if (vec > 0.25).all():
      l.append(keypoints_array_sorted[i][3])
  gps.append(l)

gps = list(filter(None, gps))
print(len(gps))

def consecutive(data, stepsize=1):
    return np.split(data, np.where(np.diff(data[:,1]) != stepsize)[0]+1)

print(len(kp_seq))
sequences=[]
seq_len=16
slide=4
c=0
for lst in kp_seq:
  c+=1
  ll=np.array(lst)
  x=[np.array(i) for i in consecutive(ll)]
  
  for tt in x:
    if(len(tt)>=seq_len):

      pp=[[tt[i:i + seq_len],tt[i+seq_len:i + 2*seq_len]] for i in range(0, len(tt)-(2*seq_len), slide)]
      sequences.extend(pp)

print(len(sequences))

array_of_seq=[]
tuples_to_save=[]
tups_x=[]
tups_y=[]
array_of_data=[]
for i in sequences:
  # print(len(i[0]),len(i[1]))
  x=np.array([p[3] for p in i[0]])
  y=np.array([p[3] for p in i[1]])
  array_of_seq.append([x,y])
  tuples_to_save.extend([(p[0],p[1],p[2]) for p in i[0]])
  tuples_to_save.extend([(p[0],p[1],p[2]) for p in i[1]])
#     tups_y.append()
  array_of_data.append([x,y,np.array([(p[0],p[1],p[2]) for p in i[0]]),np.array([(p[0],p[1],p[2]) for p in i[1]])])
  
array_of_seq=np.array(array_of_seq)
print(array_of_seq.shape)
# print(len(tuples_to_save),tuples_to_save[0])
# array_of_data[0]

4711
244
244
84
(84, 2, 16, 51)


C:\Users\bigbo\miniconda3\envs\pv-lstm\lib\site-packages\ipykernel_launcher.py:34: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [145]:
import random 
seq_out=array_of_seq
print(seq_out.shape)
sequences_all=[]
sequences_obs_speed=[]
sequences_true_speed=[]
sequences_obs_pose=[]
sequences_true_pose=[]
sequences_true_imgs=[]
sequences_obs_imgs=[]
seq_len=16
split=int(0.75*len(seq_out))
for p in range(len(seq_out)):
  outputs = []

  observed = array_of_data[p][0]
  future = array_of_data[p][1]
  obs = torch.tensor([observed[i] for i in range(0,seq_len,1)])
  true = torch.tensor([future[i] for i in range(0,seq_len,1)])
  sequences_obs_pose.append(np.round(obs.numpy(),2).tolist())
  sequences_true_pose.append(np.round(true.numpy(),2).tolist())
#   sequences_true_imgs.append(['/frames/'+'video_'+str(i[0]).rjust(4, '0')+"/"+str(i[1]).rjust(5, '0')+'.png' for i in array_of_data[p][2]])
#   sequences_obs_imgs.append(['/frames/'+'video_'+str(i[0]).rjust(4, '0')+"/"+str(i[1]).rjust(5, '0')+'.png' for i in array_of_data[p][3]])
#   
c = list(zip(sequences_obs_pose, sequences_true_pose))#,sequences_true_imgs,sequences_obs_imgs))

random.shuffle(c)

sequences_obs_pose, sequences_true_pose = zip(*c) #, sequences_true_imgs, sequences_obs_imgs

sequences_all_train=sequences_all[:split]
sequences_all_val=sequences_all[split:]

sequences_all=np.array(sequences_all)

data = {'Pose': sequences_obs_pose,#[:split]
        'Future_Pose': sequences_true_pose,#[:split]
       }
#         'Pose_image':sequences_true_imgs[:split],
#         'Future_image':sequences_obs_imgs[:split]
#         }
data_val = {'Pose': sequences_obs_pose[split:],
        'Future_Pose': sequences_true_pose[split:],
           }
#         'Pose_image':sequences_true_imgs[split:],
#         'Future_image':sequences_obs_imgs[split:]
#         }
df_train = pd.DataFrame (data, columns = ['Pose','Future_Pose'])#,'Pose_image','Future_image'])
# df_val = pd.DataFrame (data_val, columns = ['Pose','Future_Pose','Pose_image','Future_image'])

df_train.to_csv("./sequences_openpifpaf_complete.csv",index=False)
# df_val.to_csv("./sequences_openpose_wconfsc_wimage_val.csv", index=False)

print(df_train.head())

(84, 2, 16, 51)
                                                Pose  \
0  [[1731.22, 627.23, 0.65, 1735.22, 622.75, 0.63...   
1  [[1534.6, 591.08, 0.86, 1538.25, 587.28, 0.91,...   
2  [[1120.28, 697.56, 0.68, 1122.15, 694.75, 0.61...   
3  [[292.51, 709.45, 0.9, 296.83, 704.17, 0.83, 2...   
4  [[1170.75, 705.09, 0.54, 1173.13, 702.74, 0.56...   

                                         Future_Pose  
0  [[1775.1, 618.23, 0.82, 1780.46, 613.15, 0.83,...  
1  [[1576.73, 582.43, 0.75, 1580.1, 578.38, 0.77,...  
2  [[1125.55, 698.53, 0.65, 1127.6, 695.64, 0.59,...  
3  [[276.08, 708.12, 0.75, 281.11, 702.83, 0.76, ...  
4  [[1069.5, 695.04, 0.28, 1071.41, 692.14, 0.62,...  


In [149]:
print(sequences_obs_pose[0][0])

[1731.22, 627.23, 0.65, 1735.22, 622.75, 0.63, 1726.39, 623.11, 0.55, 1741.33, 623.83, 0.5, 1719.77, 624.4, 0.52, 1748.3, 644.41, 0.72, 1709.8, 645.18, 0.55, 1760.71, 674.04, 0.92, 1700.98, 688.03, 0.51, 1749.28, 643.29, 0.85, 1694.57, 722.97, 0.57, 1748.33, 715.04, 0.71, 1720.37, 716.53, 0.68, 1755.05, 775.45, 0.74, 1718.74, 775.83, 0.72, 1758.17, 827.18, 0.77, 1701.24, 826.15, 0.77]


In [192]:
np.random.shuffle(kp_train)
train_dataset=PoseDataset(kp_train)
val_dataset=PoseDataset(kp_train)
trainloader=DataLoader(train_dataset, batch_size=BATCH_SIZE)
valloader=DataLoader(val_dataset, batch_size=BATCH_SIZE)

In [210]:
# net = Autoencoder()
# device = torch.device('cuda:0')
# net.to(device)
# net = net.float()
NUM_EPOCHS = 300
BATCH_SIZE = 30
# net.load_state_dict(torch.load('model_best.weight'))

criterion = nn.L1Loss()
optimizer = optim.Adam(net.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5,verbose=True, factor=0.5)
train_loss, val_loss = train(net, trainloader, valloader, NUM_EPOCHS, scheduler)


Epoch 1 of 300, Train Loss: 77.724, Val Loss: 30.204, ADE_train : 125.811,  ADE_val: 47.361
Epoch 2 of 300, Train Loss: 77.268, Val Loss: 31.273, ADE_train : 125.119,  ADE_val: 50.251
Epoch 3 of 300, Train Loss: 76.848, Val Loss: 27.347, ADE_train : 124.749,  ADE_val: 41.752
Epoch 4 of 300, Train Loss: 76.571, Val Loss: 29.945, ADE_train : 124.053,  ADE_val: 48.083
Epoch 5 of 300, Train Loss: 74.731, Val Loss: 28.471, ADE_train : 120.241,  ADE_val: 43.242
Epoch 6 of 300, Train Loss: 77.387, Val Loss: 30.293, ADE_train : 125.471,  ADE_val: 48.296
Epoch 7 of 300, Train Loss: 76.645, Val Loss: 33.665, ADE_train : 124.245,  ADE_val: 54.728
Epoch 8 of 300, Train Loss: 74.624, Val Loss: 29.073, ADE_train : 120.739,  ADE_val: 46.294
Epoch     9: reducing learning rate of group 0 to 5.0000e-04.
Epoch 9 of 300, Train Loss: 78.249, Val Loss: 33.028, ADE_train : 126.774,  ADE_val: 52.757
Epoch 10 of 300, Train Loss: 76.607, Val Loss: 30.868, ADE_train : 123.880,  ADE_val: 48.765
Epoch 11 of 300, 

Epoch 82 of 300, Train Loss: 76.134, Val Loss: 28.971, ADE_train : 123.310,  ADE_val: 45.463
Epoch 83 of 300, Train Loss: 77.065, Val Loss: 28.970, ADE_train : 125.071,  ADE_val: 45.459
Epoch 84 of 300, Train Loss: 75.211, Val Loss: 28.979, ADE_train : 122.040,  ADE_val: 45.480
Epoch 85 of 300, Train Loss: 75.739, Val Loss: 28.976, ADE_train : 122.868,  ADE_val: 45.473
Epoch 86 of 300, Train Loss: 74.369, Val Loss: 28.978, ADE_train : 120.237,  ADE_val: 45.478
Epoch    87: reducing learning rate of group 0 to 6.1035e-08.
Epoch 87 of 300, Train Loss: 74.728, Val Loss: 28.979, ADE_train : 121.024,  ADE_val: 45.479
Epoch 88 of 300, Train Loss: 75.980, Val Loss: 28.979, ADE_train : 122.758,  ADE_val: 45.479
Epoch 89 of 300, Train Loss: 74.376, Val Loss: 28.980, ADE_train : 120.546,  ADE_val: 45.481
Epoch 90 of 300, Train Loss: 75.695, Val Loss: 28.977, ADE_train : 122.963,  ADE_val: 45.476
Epoch 91 of 300, Train Loss: 74.695, Val Loss: 28.978, ADE_train : 120.798,  ADE_val: 45.477
Epoch 92

Epoch 168 of 300, Train Loss: 75.869, Val Loss: 28.967, ADE_train : 123.158,  ADE_val: 45.455
Epoch 169 of 300, Train Loss: 77.444, Val Loss: 28.967, ADE_train : 126.182,  ADE_val: 45.455
Epoch 170 of 300, Train Loss: 74.900, Val Loss: 28.967, ADE_train : 121.454,  ADE_val: 45.455
Epoch 171 of 300, Train Loss: 74.567, Val Loss: 28.967, ADE_train : 120.421,  ADE_val: 45.455
Epoch 172 of 300, Train Loss: 76.138, Val Loss: 28.967, ADE_train : 122.968,  ADE_val: 45.455
Epoch 173 of 300, Train Loss: 74.601, Val Loss: 28.967, ADE_train : 120.643,  ADE_val: 45.455
Epoch 174 of 300, Train Loss: 76.192, Val Loss: 28.967, ADE_train : 123.890,  ADE_val: 45.455
Epoch 175 of 300, Train Loss: 76.781, Val Loss: 28.967, ADE_train : 124.346,  ADE_val: 45.455
Epoch 176 of 300, Train Loss: 75.298, Val Loss: 28.967, ADE_train : 121.358,  ADE_val: 45.454
Epoch 177 of 300, Train Loss: 76.470, Val Loss: 28.967, ADE_train : 124.336,  ADE_val: 45.455
Epoch 178 of 300, Train Loss: 75.527, Val Loss: 28.967, ADE_

Epoch 256 of 300, Train Loss: 77.222, Val Loss: 28.963, ADE_train : 124.950,  ADE_val: 45.446
Epoch 257 of 300, Train Loss: 76.243, Val Loss: 28.963, ADE_train : 123.292,  ADE_val: 45.446
Epoch 258 of 300, Train Loss: 75.234, Val Loss: 28.963, ADE_train : 122.065,  ADE_val: 45.447
Epoch 259 of 300, Train Loss: 76.066, Val Loss: 28.963, ADE_train : 123.703,  ADE_val: 45.447
Epoch 260 of 300, Train Loss: 76.494, Val Loss: 28.963, ADE_train : 124.107,  ADE_val: 45.447
Epoch 261 of 300, Train Loss: 74.617, Val Loss: 28.963, ADE_train : 121.212,  ADE_val: 45.447
Epoch 262 of 300, Train Loss: 76.540, Val Loss: 28.963, ADE_train : 123.837,  ADE_val: 45.447
Epoch 263 of 300, Train Loss: 75.332, Val Loss: 28.963, ADE_train : 121.939,  ADE_val: 45.447
Epoch 264 of 300, Train Loss: 76.796, Val Loss: 28.963, ADE_train : 124.477,  ADE_val: 45.447
Epoch 265 of 300, Train Loss: 76.295, Val Loss: 28.963, ADE_train : 123.590,  ADE_val: 45.448
Epoch 266 of 300, Train Loss: 75.465, Val Loss: 28.964, ADE_

In [211]:
torch.save(net.state_dict(), "model_temp.weight")

In [5]:
class LSTM_spat(nn.Module):
    def __init__(self, args):
        '''
           input: observed body poses and velocites 
           output: predicted poses(not trained), velocities of poses, and intentions
        '''
        super(LSTM_spat, self).__init__()
        self.encoded_size=300
         
        self.pose_encoder = nn.LSTM(input_size=self.encoded_size, hidden_size=args.hidden_size, num_layers = 3)        
        self.pose_embedding = nn.Sequential(nn.Linear(in_features=args.hidden_size, out_features=30),
                                           nn.ReLU())
        
        self.pose_decoder = nn.LSTM(input_size=self.encoded_size, hidden_size=args.hidden_size, num_layers = 3)        
        self.fc_pose   = nn.Linear(in_features=args.hidden_size, out_features=self.encoded_size)
        
        self.hardtanh = nn.Hardtanh(min_val=-1*args.hardtanh_limit,max_val=args.hardtanh_limit)
        self.relu = nn.ReLU() 
        self.softmax = nn.Softmax(dim=1)
        
        self.enc1= nn.Linear(in_features=34, out_features=300)
        self.enc2 = nn.Linear(in_features=300, out_features=self.encoded_size)
        self.dec = nn.Linear(in_features=self.encoded_size, out_features=34)
        
#         self.drop=n
        
#         self.encoder = lstm_encoder(input_size = 34, hidden_size = 1000, num_layers = 3)
#         self.decoder = lstm_decoder(input_size = 34, hidden_size = 1000, num_layers = 3)

#         self.teacher_forcing_ratio = 0.6
        
        self.args = args
        
    def forward(self, pose=None, vel=None, target_pose=None):
        
        poses=pose.permute(1,0,2)
#         target_poses=target_pose.permute(1,0,2)
        
# #         input_batch=pose
# #         outputs = torch.zeros(target_len, input_batch.shape[1], input_batch.shape[2]).to('cuda')
# #         # encoder outputs
# #         encoder_output, encoder_hidden = model.encoder(input_batch)
# #         # decoder with teacher forcing
# #         decoder_input = input_batch[-1, :, :]   # shape: (batch_size, input_size)
# #         decoder_hidden = encoder_hidden
        
# #         print(decoder_input.shape)

# #         for t in range(16): 
# #             decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden)
# #             outputs[t] = decoder_output
# #             decoder_input = decoder_output



        pose_encoded = torch.tensor([], device=self.args.device)
        pose_recs = torch.tensor([], device=self.args.device)
#         target_pose_encoded = torch.tensor([], device=self.args.device)
        
        for i in range(pose.size()[0]):
#             print(i)
            x = self.relu(self.enc1(pose[i]))
            x = self.relu(self.enc2(x))
            recreated_pose=self.relu(self.dec(x))
            pose_encoded = torch.cat((pose_encoded, x.unsqueeze(1)), dim = 1)
            pose_recs = torch.cat((pose_recs, recreated_pose.unsqueeze(1)), dim = 1)
        
# #         vel_encoded = torch.tensor([], device=self.args.device)
       
# #         for i in range(vel.size()[0]):
# # #             print(i)
# #             x = self.enc1(vel[i])
# #             x = self.enc2(x)
# #             vel_encoded = torch.cat((vel_encoded, x.unsqueeze(1)), dim = 1)
        
# #         for i in range(target_pose.size()[0]):
# # #             print(i)
# #             x = self.relu(self.enc1(self.relu(target_pose[i])))
# #             x = self.relu(self.enc2(x))
# #             target_pose_encoded = torch.cat((target_pose_encoded, x.unsqueeze(1)), dim = 1)
        
# #         print(pose_encoded.size())
# #         _, (hidden_vel, cell_vel) = self.vel_encoder()
# #         hidden_vel = hidden_vel.squeeze(0)
# #         cell_vel = cell_vel.squeeze(0)

#         print(pose_encoded.shape)
        
        _, (hidden_st) = self.pose_encoder(pose_encoded)
#         hidden_pose = hidden_pose#.squeeze(0)
#         cell_pose = cell_pose#.squeeze(0)
        
        outputs = []
        pose_outputs   = torch.tensor([], device=self.args.device)
        PoseDec_inp = pose_encoded.permute(1,0,2)[:,-1,:].unsqueeze(0)
        
#         print(PoseDec_inp.size())
        hidden_dec=hidden_st
#         cell_dec=cell_pose
#         print(PoseDec_inp.size(),hidden_dec.shape)
        
        for i in range(self.args.output//self.args.skip):
            pose_output_l, hidden_dec = self.pose_decoder(PoseDec_inp, hidden_dec)
#             print(hidden_dec)

            pose_output_encoded  = self.fc_pose(pose_output_l.squeeze(0))
            pose_output= self.dec(pose_output_encoded)
            pose_outputs = torch.cat((pose_outputs, pose_output.unsqueeze(1)), dim = 1)
            PoseDec_inp  = pose_output_encoded.detach().unsqueeze(0)
        
#         if random.random() < self.teacher_forcing_ratio:
#             for i in range(self.args.output//self.args.skip):
#                 hidden_dec, cell_dec = self.pose_decoder(PoseDec_inp, (hidden_dec, cell_dec))
#                 pose_output_encoded  = self.fc_pose(hidden_dec)
#                 pose_output= self.relu(self.dec(pose_output_encoded))
#                 pose_outputs = torch.cat((pose_outputs, pose_output.unsqueeze(1)), dim = 1)
#                 PoseDec_inp  = target_pose_encoded[i,:,:]

#                 # predict recursively 
#         else:
#             for i in range(self.args.output//self.args.skip):
#                 hidden_dec, cell_dec = self.pose_decoder(PoseDec_inp, (hidden_dec, cell_dec))
#                 pose_output_encoded  = self.fc_pose(hidden_dec)
#                 pose_output= self.relu(self.dec(pose_output_encoded))
#                 pose_outputs = torch.cat((pose_outputs, pose_output.unsqueeze(1)), dim = 1)
#                 PoseDec_inp  = pose_output_encoded.detach()
        
#         print(pose_outputs.shape)
#         pose_outputs=pose_outputs.permute(1,0,2)
        outputs.append(pose_outputs)
        outputs.append(pose_recs.permute(1,0,2))

            
        return tuple(outputs)



In [77]:
class args():
    def __init__(self):
        self.loader_workers = 1
        self.loader_shuffle = True
        self.pin_memory     = False
        self.device         = 'cuda'
        self.batch_size     = 50
        self.n_epochs       = 1000
        self.hidden_size    = 1000
        self.hardtanh_limit = 100
        self.input  = 16
        self.output = 16
        self.stride = 16
        self.skip   = 1
        self.lr = 0.01
args = args()


mse = nn.MSELoss()
l1e = nn.L1Loss()
train_s_scores = []
train_pose_scores=[]
val_pose_scores=[]
train_c_scores = []
val_s_scores   = []
val_c_scores   = []

# train_loader=data_loader(args,"train")
# val_loader=data_loader(args,"val")

train_loader=data_loader(args,"train","sequences_openpifpaf_")
val_loader=data_loader(args,"val","sequences_openpifpaf_")


Loading train
Pose loaded
Future_Pose loaded
******************************
Loading val
Pose loaded
Future_Pose loaded
******************************


In [38]:
net = LSTM_spat(args).to(args.device)
# optimizer = optim.Adam(net.parameters(), lr=args.lr)
# scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=15, 
#                                                  threshold = 1e-8, verbose=True)
state_dict = torch.load('model_best.weight')
# state_dict = torch.load('model_temp.weight')

# print(state_dict)
with torch.no_grad():
    net.enc1.weight.copy_(state_dict['enc1.weight'])
    net.enc1.bias.copy_(state_dict['enc1.bias'])
    net.enc2.weight.copy_(state_dict['enc2.weight'])
    net.enc2.bias.copy_(state_dict['enc2.bias'])
    net.dec.weight.copy_(state_dict['dec4.weight'])
    net.dec.bias.copy_(state_dict['dec4.bias'])
    net.enc1.requires_grad = False
    net.enc2.requires_grad = False
    net.dec.requires_grad = False
    

In [39]:

optimizer = optim.Adam(net.parameters(), lr= 0.001)
# optimizer = optim.Adadelta(net.parameters(),lr= 0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=8, 
                                                 threshold = 1e-12, verbose=True)
print('='*100)
print('Training ...')

train_p_scores=[]
val_p_scores=[]

for epoch in range(500):
    start = time.time()
    
    avg_epoch_train_p_loss   = 0
    avg_epoch_val_p_loss     = 0 
    ade  = 0
    fde  = 0
    ade_train  = 0
    fde_train  = 0
    counter = 0
    net.train()
    for idx, (obs_s, target_s, obs_pose, target_pose) in enumerate(train_loader):
        counter += 1        
        
        
        obs_pose    = obs_pose.to(device='cuda')
        target_pose = target_pose.to(device='cuda')
 
        net.zero_grad()
    
        (pose_preds,pose_recs) = net(pose=obs_pose, target_pose=target_pose)

        loss  = l1e(pose_preds, target_pose)#+l1e(pose_recs, obs_pose)
    
        ade_train += float(ADE_c(pose_preds, target_pose))
        fde_train += float(FDE_c(pose_preds, target_pose))
        
    
        loss.backward()
        optimizer.step()
        
    
        avg_epoch_train_p_loss += float(loss)

    avg_epoch_train_p_loss /= counter
    train_p_scores.append(avg_epoch_train_p_loss)
    ade_train  /= counter
    fde_train  /= counter   
    
    
  
    counter=0
    net.eval()
    for idx, (obs_s, target_s, obs_pose, target_pose) in enumerate(val_loader):
        counter+=1
       
        obs_pose    = obs_pose.to(device='cuda')
        target_pose = target_pose.to(device='cuda')
        
        with torch.no_grad():
            (pose_preds,pose_recs) = net(pose=obs_pose, target_pose=target_pose)
          
            val_loss  = l1e(pose_preds, target_pose)#+l1e(pose_recs, obs_pose)
            avg_epoch_val_p_loss += float(val_loss)
            
            ade += float(ADE_c(pose_preds, target_pose))
            fde += float(FDE_c(pose_preds, target_pose))

        
    avg_epoch_val_p_loss /= counter
    val_p_scores.append(avg_epoch_val_p_loss)
    
    ade  /= counter
    fde  /= counter     
   
    
    
    scheduler.step(avg_epoch_val_p_loss)
    
    print('e:', epoch,'| loss: %.2f'%avg_epoch_train_p_loss,'| val_loss: %.2f'% avg_epoch_val_p_loss, '| tpose: %.2f'% avg_epoch_train_p_loss, '| vpose: %.2f'% avg_epoch_val_p_loss, '| ade_train: %.2f'% ade_train, '| ade_val: %.2f'% ade, '| fde_train: %.2f'% fde_train,'| fde_val: %.2f'% fde,
          '| t:%.2f'%(time.time()-start))


print('='*100) 
# print('Saving ...')
# torch.save(net.state_dict(), args.model_path)
print('Done !')

Training ...
e: 0 | loss: 851.31 | val_loss: 862.31 | tpose: 851.31 | vpose: 862.31 | ade_train: 1257.86 | ade_val: 1286.46 | fde_train: 1256.09 | fde_val: 1284.93 | t:0.70
e: 1 | loss: 794.99 | val_loss: 802.86 | tpose: 794.99 | vpose: 802.86 | ade_train: 1181.16 | ade_val: 1206.50 | fde_train: 1178.23 | fde_val: 1205.55 | t:0.67
e: 2 | loss: 735.87 | val_loss: 745.27 | tpose: 735.87 | vpose: 745.27 | ade_train: 1101.59 | ade_val: 1129.24 | fde_train: 1099.40 | fde_val: 1128.64 | t:0.64
e: 3 | loss: 677.12 | val_loss: 682.97 | tpose: 677.12 | vpose: 682.97 | ade_train: 1021.59 | ade_val: 1044.27 | fde_train: 1020.01 | fde_val: 1043.99 | t:0.65
e: 4 | loss: 612.64 | val_loss: 614.84 | tpose: 612.64 | vpose: 614.84 | ade_train: 933.44 | ade_val: 953.09 | fde_train: 932.55 | fde_val: 953.28 | t:0.64
e: 5 | loss: 544.68 | val_loss: 543.89 | tpose: 544.68 | vpose: 543.89 | ade_train: 841.84 | ade_val: 859.43 | fde_train: 841.48 | fde_val: 860.52 | t:0.63
e: 6 | loss: 478.15 | val_loss: 478

e: 51 | loss: 196.05 | val_loss: 202.40 | tpose: 196.05 | vpose: 202.40 | ade_train: 358.93 | ade_val: 368.17 | fde_train: 374.05 | fde_val: 391.47 | t:0.65
e: 52 | loss: 195.95 | val_loss: 202.37 | tpose: 195.95 | vpose: 202.37 | ade_train: 358.83 | ade_val: 368.12 | fde_train: 374.15 | fde_val: 391.42 | t:0.65
e: 53 | loss: 196.17 | val_loss: 202.38 | tpose: 196.17 | vpose: 202.38 | ade_train: 359.16 | ade_val: 368.15 | fde_train: 374.38 | fde_val: 391.45 | t:0.65
e: 54 | loss: 196.08 | val_loss: 202.40 | tpose: 196.08 | vpose: 202.40 | ade_train: 359.07 | ade_val: 368.17 | fde_train: 374.37 | fde_val: 391.48 | t:0.65
e: 55 | loss: 195.96 | val_loss: 202.41 | tpose: 195.96 | vpose: 202.41 | ade_train: 358.88 | ade_val: 368.18 | fde_train: 374.13 | fde_val: 391.49 | t:0.64
e: 56 | loss: 196.05 | val_loss: 202.45 | tpose: 196.05 | vpose: 202.45 | ade_train: 358.98 | ade_val: 368.27 | fde_train: 374.05 | fde_val: 391.58 | t:0.65
e: 57 | loss: 196.19 | val_loss: 202.48 | tpose: 196.19 | 

e: 102 | loss: 196.79 | val_loss: 202.51 | tpose: 196.79 | vpose: 202.51 | ade_train: 360.37 | ade_val: 368.40 | fde_train: 375.45 | fde_val: 391.72 | t:0.65
Epoch   104: reducing learning rate of group 0 to 1.9531e-06.
e: 103 | loss: 196.05 | val_loss: 202.52 | tpose: 196.05 | vpose: 202.52 | ade_train: 359.05 | ade_val: 368.40 | fde_train: 374.23 | fde_val: 391.72 | t:0.65
e: 104 | loss: 195.89 | val_loss: 202.52 | tpose: 195.89 | vpose: 202.52 | ade_train: 358.73 | ade_val: 368.40 | fde_train: 373.81 | fde_val: 391.72 | t:0.65
e: 105 | loss: 195.88 | val_loss: 202.52 | tpose: 195.88 | vpose: 202.52 | ade_train: 358.71 | ade_val: 368.40 | fde_train: 373.69 | fde_val: 391.72 | t:0.68
e: 106 | loss: 196.42 | val_loss: 202.52 | tpose: 196.42 | vpose: 202.52 | ade_train: 359.69 | ade_val: 368.40 | fde_train: 374.92 | fde_val: 391.72 | t:0.73
e: 107 | loss: 196.22 | val_loss: 202.52 | tpose: 196.22 | vpose: 202.52 | ade_train: 359.27 | ade_val: 368.40 | fde_train: 374.31 | fde_val: 391.72

e: 152 | loss: 196.69 | val_loss: 202.52 | tpose: 196.69 | vpose: 202.52 | ade_train: 360.19 | ade_val: 368.40 | fde_train: 375.29 | fde_val: 391.72 | t:0.77
e: 153 | loss: 196.16 | val_loss: 202.52 | tpose: 196.16 | vpose: 202.52 | ade_train: 359.21 | ade_val: 368.40 | fde_train: 374.30 | fde_val: 391.72 | t:0.73
e: 154 | loss: 196.25 | val_loss: 202.52 | tpose: 196.25 | vpose: 202.52 | ade_train: 359.27 | ade_val: 368.40 | fde_train: 374.25 | fde_val: 391.72 | t:0.74
e: 155 | loss: 196.20 | val_loss: 202.52 | tpose: 196.20 | vpose: 202.52 | ade_train: 359.18 | ade_val: 368.40 | fde_train: 374.20 | fde_val: 391.72 | t:0.69
e: 156 | loss: 196.17 | val_loss: 202.52 | tpose: 196.17 | vpose: 202.52 | ade_train: 359.18 | ade_val: 368.40 | fde_train: 374.15 | fde_val: 391.72 | t:0.76
Epoch   158: reducing learning rate of group 0 to 3.0518e-08.
e: 157 | loss: 196.13 | val_loss: 202.52 | tpose: 196.13 | vpose: 202.52 | ade_train: 359.16 | ade_val: 368.40 | fde_train: 374.40 | fde_val: 391.72

e: 204 | loss: 196.47 | val_loss: 202.52 | tpose: 196.47 | vpose: 202.52 | ade_train: 359.79 | ade_val: 368.40 | fde_train: 374.91 | fde_val: 391.72 | t:0.77
e: 205 | loss: 195.97 | val_loss: 202.52 | tpose: 195.97 | vpose: 202.52 | ade_train: 358.89 | ade_val: 368.40 | fde_train: 374.06 | fde_val: 391.72 | t:0.78
e: 206 | loss: 196.03 | val_loss: 202.52 | tpose: 196.03 | vpose: 202.52 | ade_train: 359.07 | ade_val: 368.40 | fde_train: 374.17 | fde_val: 391.72 | t:0.72
e: 207 | loss: 196.34 | val_loss: 202.52 | tpose: 196.34 | vpose: 202.52 | ade_train: 359.53 | ade_val: 368.40 | fde_train: 374.66 | fde_val: 391.72 | t:0.78
e: 208 | loss: 196.07 | val_loss: 202.52 | tpose: 196.07 | vpose: 202.52 | ade_train: 359.01 | ade_val: 368.40 | fde_train: 374.18 | fde_val: 391.72 | t:0.75
e: 209 | loss: 195.69 | val_loss: 202.52 | tpose: 195.69 | vpose: 202.52 | ade_train: 358.39 | ade_val: 368.40 | fde_train: 373.48 | fde_val: 391.72 | t:0.70
e: 210 | loss: 196.27 | val_loss: 202.52 | tpose: 19

KeyboardInterrupt: 

In [94]:
import numpy as np
import random
import os, errno
import sys
from tqdm import trange

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F


class lstm_encoder(nn.Module):
    ''' Encodes time-series sequence '''

    def __init__(self, input_size, hidden_size, num_layers = 1):
        
        '''
        : param input_size:     the number of features in the input X
        : param hidden_size:    the number of features in the hidden state h
        : param num_layers:     number of recurrent layers (i.e., 2 means there are
        :                       2 stacked LSTMs)
        '''
        
        super(lstm_encoder, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # define LSTM layer
        self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size,
                            num_layers = num_layers)

    def forward(self, x_input):
        
        '''
        : param x_input:               input of shape (seq_len, # in batch, input_size)
        : return lstm_out, hidden:     lstm_out gives all the hidden states in the sequence;
        :                              hidden gives the hidden state and cell state for the last
        :                              element in the sequence 
        '''
        
        lstm_out, self.hidden = self.lstm(x_input.view(x_input.shape[0], x_input.shape[1], self.input_size))
        
        return lstm_out, self.hidden     
    
    def init_hidden(self, batch_size):
        
        '''
        initialize hidden state
        : param batch_size:    x_input.shape[1]
        : return:              zeroed hidden state and cell state 
        '''
        
        return (torch.zeros(self.num_layers, batch_size, self.hidden_size),
                torch.zeros(self.num_layers, batch_size, self.hidden_size))


class lstm_decoder(nn.Module):
    ''' Decodes hidden state output by encoder '''
    
    def __init__(self, input_size, hidden_size, num_layers = 1):

        '''
        : param input_size:     the number of features in the input X
        : param hidden_size:    the number of features in the hidden state h
        : param num_layers:     number of recurrent layers (i.e., 2 means there are
        :                       2 stacked LSTMs)
        '''
        
        super(lstm_decoder, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size,
                            num_layers = num_layers)
        self.linear = nn.Linear(hidden_size, input_size)           

    def forward(self, x_input, encoder_hidden_states):
        
        '''        
        : param x_input:                    should be 2D (batch_size, input_size)
        : param encoder_hidden_states:      hidden states
        : return output, hidden:            output gives all the hidden states in the sequence;
        :                                   hidden gives the hidden state and cell state for the last
        :                                   element in the sequence 
 
        '''
        
        lstm_out, self.hidden = self.lstm(x_input.unsqueeze(0), encoder_hidden_states)
        output = self.linear(lstm_out.squeeze(0))     
        
        return output, self.hidden

class lstm_seq2seq(nn.Module):
    ''' train LSTM encoder-decoder and make predictions '''
    
    def __init__(self, input_size, hidden_size, num_layers):

        '''
        : param input_size:     the number of expected features in the input X
        : param hidden_size:    the number of features in the hidden state h
        '''

        super(lstm_seq2seq, self).__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size

        self.encoder = lstm_encoder(input_size = input_size, hidden_size = hidden_size, num_layers = num_layers)
        self.decoder = lstm_decoder(input_size = input_size, hidden_size = hidden_size, num_layers = num_layers)


    def train_model(self, input_tensor, target_tensor, n_epochs, target_len, batch_size, training_prediction = 'recursive', teacher_forcing_ratio = 0.5, learning_rate = 0.01, dynamic_tf = False):
        
        '''
        train lstm encoder-decoder
        
        : param input_tensor:              input data with shape (seq_len, # in batch, number features); PyTorch tensor    
        : param target_tensor:             target data with shape (seq_len, # in batch, number features); PyTorch tensor
        : param n_epochs:                  number of epochs 
        : param target_len:                number of values to predict 
        : param batch_size:                number of samples per gradient update
        : param training_prediction:       type of prediction to make during training ('recursive', 'teacher_forcing', or
        :                                  'mixed_teacher_forcing'); default is 'recursive'
        : param teacher_forcing_ratio:     float [0, 1) indicating how much teacher forcing to use when
        :                                  training_prediction = 'teacher_forcing.' For each batch in training, we generate a random
        :                                  number. If the random number is less than teacher_forcing_ratio, we use teacher forcing.
        :                                  Otherwise, we predict recursively. If teacher_forcing_ratio = 1, we train only using
        :                                  teacher forcing.
        : param learning_rate:             float >= 0; learning rate
        : param dynamic_tf:                use dynamic teacher forcing (True/False); dynamic teacher forcing
        :                                  reduces the amount of teacher forcing for each epoch
        : return losses:                   array of loss function for each epoch
        '''
        
        # initialize array of losses 
        losses = np.full(n_epochs, np.nan)

        optimizer = optim.Adam(self.parameters(), lr = learning_rate)
        criterion = nn.L1Loss()
#         criterion = nn.MSELoss()

        # calculate number of batch iterations
        n_batches = int(input_tensor.shape[1] / batch_size)

        with trange(n_epochs) as tr:
            for it in tr:
                
                batch_loss = 0.
                batch_loss_tf = 0.
                batch_loss_no_tf = 0.
                num_tf = 0
                num_no_tf = 0

                for b in range(n_batches):
                    # select data 
                    input_batch = input_tensor[:, b: b + batch_size, :]
                    target_batch = target_tensor[:, b: b + batch_size, :]

                    # outputs tensor
                    outputs = torch.zeros(target_len, batch_size, input_batch.shape[2])

                    # initialize hidden state
                    encoder_hidden = self.encoder.init_hidden(batch_size)

                    # zero the gradient
                    optimizer.zero_grad()

                    # encoder outputs
                    encoder_output, encoder_hidden = self.encoder(input_batch)

                    # decoder with teacher forcing
                    decoder_input = input_batch[-1, :, :]   # shape: (batch_size, input_size)
                    decoder_hidden = encoder_hidden

                    if training_prediction == 'recursive':
                        # predict recursively
                        for t in range(target_len): 
                            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden)
                            outputs[t] = decoder_output
                            decoder_input = decoder_output

                    if training_prediction == 'teacher_forcing':
                        # use teacher forcing
                        if random.random() < teacher_forcing_ratio:
                            for t in range(target_len): 
                                decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden)
                                outputs[t] = decoder_output
                                decoder_input = target_batch[t, :, :]

                        # predict recursively 
                        else:
                            for t in range(target_len): 
                                decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden)
                                outputs[t] = decoder_output
                                decoder_input = decoder_output

                    if training_prediction == 'mixed_teacher_forcing':
                        # predict using mixed teacher forcing
                        for t in range(target_len):
                            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden)
                            outputs[t] = decoder_output
                            
                            # predict with teacher forcing
                            if random.random() < teacher_forcing_ratio:
                                decoder_input = target_batch[t, :, :]
                            
                            # predict recursively 
                            else:
                                decoder_input = decoder_output

                    # compute the loss 
                    loss = criterion(outputs, target_batch)
                    batch_loss += loss.item()
                    
                    # backpropagation
                    loss.backward()
                    optimizer.step()

                # loss for epoch 
                batch_loss /= n_batches 
                losses[it] = batch_loss

                # dynamic teacher forcing
                if dynamic_tf and teacher_forcing_ratio > 0:
                    teacher_forcing_ratio = teacher_forcing_ratio - 0.02 

                # progress bar 
                tr.set_postfix(loss="{0:.3f}".format(batch_loss))
                    
        return losses

    def predict(self, input_tensor, target_len):
        
        '''
        : param input_tensor:      input data (seq_len, input_size); PyTorch tensor 
        : param target_len:        number of target values to predict 
        : return np_outputs:       np.array containing predicted values; prediction done recursively 
        '''

        # encode input_tensor
        input_tensor = input_tensor.unsqueeze(1)     # add in batch size of 1
        encoder_output, encoder_hidden = self.encoder(input_tensor)

        # initialize tensor for predictions
        outputs = torch.zeros(target_len, input_tensor.shape[2])

        # decode input_tensor
        decoder_input = input_tensor[-1, :, :]
        decoder_hidden = encoder_hidden
        
        for t in range(target_len):
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden)
            outputs[t] = decoder_output.squeeze(0)
            decoder_input = decoder_output
            
        np_outputs = outputs.detach().numpy()
        
        return np_output
    

    
    
model = lstm_seq2seq(input_size = 34, hidden_size = 1000, num_layers=3)
device = torch.device('cuda:0')
# print(device)
model.to(device)

# loss = model.train_model(X_train, Y_train, n_epochs = 50, target_len = 16, batch_size = 50, training_prediction = 'mixed_teacher_forcing', teacher_forcing_ratio = 0.6, learning_rate = 0.01, dynamic_tf = False)


lstm_seq2seq(
  (encoder): lstm_encoder(
    (lstm): LSTM(34, 1000, num_layers=3)
  )
  (decoder): lstm_decoder(
    (lstm): LSTM(34, 1000, num_layers=3)
    (linear): Linear(in_features=1000, out_features=34, bias=True)
  )
)

In [114]:
train_loader=data_loader(args,"train","sequences_openpifpaf_thres_9_")
val_loader=data_loader(args,"val","sequences_openpifpaf_thres_9_")

Loading train
Pose loaded
Future_Pose loaded
******************************
Loading val
Pose loaded
Future_Pose loaded
******************************


In [95]:
n_epochs = 200
target_len = 16
batch_size = 60
training_prediction = 'recursive'
teacher_forcing_ratio = 0.4
learning_rate = 0.01
dynamic_tf = True
# initialize array of losses 
losses = np.full(n_epochs, np.nan)
losses_val = np.full(n_epochs, np.nan)

optimizer = optim.Adam(model.parameters(), lr = learning_rate)
criterion = nn.L1Loss()
#         criterion = nn.MSELoss()
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=10, 
                                                 threshold = 1e-12, verbose=True)
# calculate number of batch iterations
# n_batches = int(input_tensor.shape[1] / batch_size)

with trange(n_epochs) as tr:
    
    for it in tr:
        ade=0.0
        val_ade=0.0
        fde=0.0
        val_fde=0.0
        
        batch_loss = 0.
        batch_loss_val = 0.
        batch_loss_tf = 0.
        batch_loss_no_tf = 0.
        num_tf = 0
        num_no_tf = 0
        counter=0
        
        model.train()
        for idx, (obs_s, target_s, obs_pose, target_pose) in enumerate(train_loader):
            obs_s = obs_s.to(device)
            target_s = target_s.to(device)
            obs_pose = obs_pose.to(device)
            target_pose = target_pose.to(device)
            
            counter+=1
            # select data 
            input_batch = obs_s.permute(1,0,2)
            target_batch = target_s.permute(1,0,2)

            # outputs tensor
            outputs = torch.zeros(target_len, input_batch.shape[1], input_batch.shape[2]).to(device)

            # initialize hidden state
            encoder_hidden = model.encoder.init_hidden(batch_size)

            # zero the gradient
            optimizer.zero_grad()

            # encoder outputs
            encoder_output, encoder_hidden = model.encoder(input_batch)

            # decoder with teacher forcing
            decoder_input = input_batch[-1, :, :]   # shape: (batch_size, input_size)
            decoder_hidden = encoder_hidden

            if training_prediction == 'recursive':
                # predict recursively
                for t in range(target_len): 
                    decoder_output, decoder_hidden = model.decoder(decoder_input, decoder_hidden)
                    outputs[t] = decoder_output
                    decoder_input = decoder_output

            if training_prediction == 'teacher_forcing':
                # use teacher forcing
                if random.random() < teacher_forcing_ratio:
                    for t in range(target_len): 
                        decoder_output, decoder_hidden = model.decoder(decoder_input, decoder_hidden)
                        outputs[t] = decoder_output
                        decoder_input = target_batch[t, :, :]

                # predict recursively 
                else:
                    for t in range(target_len): 
                        decoder_output, decoder_hidden = model.decoder(decoder_input, decoder_hidden)
                        outputs[t] = decoder_output
                        decoder_input = decoder_output

            if training_prediction == 'mixed_teacher_forcing':
                # predict using mixed teacher forcing
                for t in range(target_len):
                    decoder_output, decoder_hidden = model.decoder(decoder_input, decoder_hidden)
                    outputs[t] = decoder_output

                    # predict with teacher forcing
                    if random.random() < teacher_forcing_ratio:
                        decoder_input = target_batch[t, :, :]

                    # predict recursively 
                    else:
                        decoder_input = decoder_output

            # compute the loss 
#             print(outputs.shape)
#             copy=torch.tensor(outputs)
            out_pose=speed2pos(outputs.clone().detach().permute(1,0,2),obs_pose)
            loss = criterion(outputs, target_s.permute(1,0,2))+criterion(out_pose, target_pose)
            
            ade += float(ADE_c(out_pose, target_pose))
            fde += float(FDE_c(out_pose, target_pose))
            batch_loss += loss.item()
            
            
            
            # backpropagation
            loss.backward()
            optimizer.step()

        # loss for epoch 
        batch_loss /= counter
        losses[it] = batch_loss
        scheduler.step(batch_loss)
        ade/=counter
        fde/=counter

        # dynamic teacher forcing
        if dynamic_tf and teacher_forcing_ratio > 0:
            teacher_forcing_ratio = teacher_forcing_ratio - 0.02 
         
        counter=0
        model.eval()
        for idx, (obs_s, target_s, obs_pose, target_pose) in enumerate(val_loader):
            obs_s = obs_s.to(device)
            target_s = target_s.to(device)
            obs_pose = obs_pose.to(device)
            target_pose = target_pose.to(device)
            
            counter+=1
            # select data 
            input_batch = obs_s.permute(1,0,2)
            target_batch = target_s.permute(1,0,2)

            # outputs tensor
            outputs = torch.zeros(target_len, input_batch.shape[1], input_batch.shape[2]).to(device)

            # initialize hidden state
            encoder_hidden = model.encoder.init_hidden(batch_size)

            # encoder outputs
            encoder_output, encoder_hidden = model.encoder(input_batch)

            # decoder with teacher forcing
            decoder_input = input_batch[-1, :, :]   # shape: (batch_size, input_size)
            decoder_hidden = encoder_hidden
            
            for t in range(target_len): 
                    decoder_output, decoder_hidden = model.decoder(decoder_input, decoder_hidden)
                    outputs[t] = decoder_output
                    decoder_input = decoder_output

            out_pose=speed2pos(outputs.clone().detach().permute(1,0,2),obs_pose)
            loss = criterion(outputs, target_s.permute(1,0,2))+criterion(out_pose, target_pose)
            
            val_ade += float(ADE_c(out_pose, target_pose))
            val_fde += float(FDE_c(out_pose, target_pose))
            batch_loss_val += loss.item()
            
        
        # loss for epoch 
        batch_loss_val /= counter
        losses_val[it] = batch_loss_val

        val_ade/=counter
        val_fde/=counter
        # progress bar 
        tr.set_postfix(loss="{0:.3f}".format(batch_loss),val_loss="{0:.3f}".format(batch_loss_val),ade_train="{0:.3f}".format(ade),ade_val="{0:.3f}".format(val_ade),fde_train="{0:.3f}".format(fde),fde_val="{0:.3f}".format(val_fde))

        

 17%|▏| 34/200 [07:25<36:02, 13.03s/it, ade_train=47.959, ade_val=50.899, fde_train=72.369, fde_val=77.784, loss=45.869

Epoch    35: reducing learning rate of group 0 to 5.0000e-03.


 42%|▍| 84/200 [18:18<25:09, 13.01s/it, ade_train=44.782, ade_val=48.550, fde_train=64.805, fde_val=69.816, loss=43.745

Epoch    85: reducing learning rate of group 0 to 2.5000e-03.


 74%|▋| 149/200 [32:40<11:40, 13.74s/it, ade_train=41.830, ade_val=45.947, fde_train=58.395, fde_val=66.760, loss=41.76

Epoch   150: reducing learning rate of group 0 to 1.2500e-03.


100%|█| 200/200 [43:58<00:00, 13.19s/it, ade_train=39.712, ade_val=47.172, fde_train=55.190, fde_val=69.253, loss=40.29


In [ ]:
torch.save(model.state_dict(), './seq2seq_openpose.model')

In [156]:
keypoints=np.array(sequences_obs_pose)
kp_train=np.delete(keypoints, list(range(2, keypoints.shape[2], 3)), axis=2)
kp_train=kp_train.reshape(kp_train.shape[0],-1)
print(kp_train.shape)
train_dataset=PoseDataset(kp_train[:82])
val_dataset=PoseDataset(kp_train[82:])


(84, 544)


In [86]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import torchvision
import torchvision.transforms as transforms


class OO_LSTM_vel(nn.Module):
    def __init__(self, args):
        '''
           input: observed body poses and velocites 
           output: predicted poses(not trained), velocities of poses, and intentions
        '''
        super(OO_LSTM_vel, self).__init__()
         
        self.encoded_size=300
        self.pose_encoder = nn.LSTM(input_size=self.encoded_size, hidden_size=args.hidden_size)
        self.vel_encoder = nn.LSTM(input_size=self.encoded_size, hidden_size=args.hidden_size)
        
        self.pose_embedding = nn.Sequential(nn.Linear(in_features=args.hidden_size, out_features=self.encoded_size),
                                           nn.ReLU())
        
        #self.pose_decoder = nn.LSTMCell(input_size=34, hidden_size=args.hidden_size)
        self.vel_decoder = nn.LSTMCell(input_size=self.encoded_size, hidden_size=args.hidden_size)

      
        self.fc_vel    = nn.Linear(in_features=args.hidden_size, out_features=self.encoded_size)
        #self.fc_pose   = nn.Linear(in_features=args.hidden_size, out_features=34)
        
        self.hardtanh = nn.Hardtanh(min_val=-1*args.hardtanh_limit,max_val=args.hardtanh_limit)
        self.relu = nn.ReLU() 
        self.softmax = nn.Softmax(dim=1)
        
        self.enc1= nn.Linear(in_features=34, out_features=300)
        self.enc2 = nn.Linear(in_features=300, out_features=self.encoded_size)
        self.encp1= nn.Linear(in_features=34, out_features=300)
        self.encp2 = nn.Linear(in_features=300, out_features=self.encoded_size)
        self.dec = nn.Linear(in_features=self.encoded_size, out_features=34)
        
        self.args = args
        
    def forward(self, pose=None, vel=None):
        
        vel_encoded = torch.tensor([], device=self.args.device)

        for i in range(vel.size()[0]):
            x = self.relu(self.enc1(self.relu(vel[i])))
            x = self.relu(self.enc2(x))
            vel_encoded = torch.cat((vel_encoded, x.unsqueeze(1)), dim = 1)
        
        pose_encoded = torch.tensor([], device=self.args.device)

        for i in range(pose.size()[0]):
            x = self.relu(self.encp1(pose[i]))
            x = self.relu(self.encp2(x))
            pose_encoded = torch.cat((pose_encoded, x.unsqueeze(1)), dim = 1)
            
        
        _, (hidden_vel, cell_vel) = self.vel_encoder(vel_encoded)
        hidden_vel = hidden_vel.squeeze(0)
        cell_vel = cell_vel.squeeze(0)


        _, (hidden_pose, cell_pose) = self.pose_encoder(pose_encoded)
        hidden_pose = hidden_pose.squeeze(0)
        cell_pose = cell_pose.squeeze(0)
        
        outputs = []
        
        vel_outputs    = torch.tensor([], device=self.args.device)
        #pose_outputs   = torch.tensor([], device=self.args.device)
#         print(pose.shape)
#         print(vel.shape)
#         print(pose_encoded.shape)
#         print(vel_encoded.shape)
#         print(vel_encoded.shape)
        vel_encoded=vel_encoded.permute(1,0,2)
        pose_encoded=pose_encoded.permute(1,0,2)
        VelDec_inp = vel_encoded[:,-1,:]
#         print(VelDec_inp.shape)
      
        hidden_dec = hidden_vel+hidden_pose 
        cell_dec = cell_vel+cell_pose 
        
#         print(hidden_dec.shape)
        for i in range(self.args.output//self.args.skip):
            hidden_dec, cell_dec = self.vel_decoder(VelDec_inp, (hidden_dec, cell_dec))
#             print(hidden_dec.shape)
            vel_output_encoded  = self.hardtanh(self.fc_vel(hidden_dec))
            vel_output = self.relu(self.dec(vel_output_encoded))
            vel_outputs = torch.cat((vel_outputs, vel_output.unsqueeze(1)), dim = 1)
            VelDec_inp  = vel_output_encoded.detach()
            
        outputs.append(vel_outputs)     
        
        #         if random.random() < self.teacher_forcing_ratio:
#             for i in range(self.args.output//self.args.skip):
#                 hidden_dec, cell_dec = self.pose_decoder(PoseDec_inp, (hidden_dec, cell_dec))
#                 pose_output_encoded  = self.fc_pose(hidden_dec)
#                 pose_output= self.relu(self.dec(pose_output_encoded))
#                 pose_outputs = torch.cat((pose_outputs, pose_output.unsqueeze(1)), dim = 1)
#                 PoseDec_inp  = target_pose_encoded[i,:,:]

#                 # predict recursively 
#         else:
#             for i in range(self.args.output//self.args.skip):
#                 hidden_dec, cell_dec = self.pose_decoder(PoseDec_inp, (hidden_dec, cell_dec))
#                 pose_output_encoded  = self.fc_pose(hidden_dec)
#                 pose_output= self.relu(self.dec(pose_output_encoded))
#                 pose_outputs = torch.cat((pose_outputs, pose_output.unsqueeze(1)), dim = 1)
#                 PoseDec_inp  = pose_output_encoded.detach()
        #outputs.append(pose_outputs)

            
        return tuple(outputs)



In [87]:
class args():
    def __init__(self):
        self.loader_workers = 1
        self.loader_shuffle = True
        self.pin_memory     = False
        self.device         = 'cuda'
        self.batch_size     = 50
        self.n_epochs       = 1000
        self.hidden_size    = 1000
        self.hardtanh_limit = 100
        self.input  = 16
        self.output = 16
        self.stride = 16
        self.skip   = 1
        self.lr = 0.01
args = args()


mse = nn.MSELoss()
l1e = nn.L1Loss()
train_s_scores = []
train_pose_scores=[]
val_pose_scores=[]
train_c_scores = []
val_s_scores   = []
val_c_scores   = []

train_loader=data_loader(args,"train","sequences_openpifpaf_")
val_loader=data_loader(args,"val","sequences_openpifpaf_")
# train_loader=data_loader(args,"train","sequences_openpifpaf_single_")
# val_loader=data_loader(args,"val","sequences_openpifpaf_single_")

Loading train
Pose loaded
Future_Pose loaded
******************************
Loading val
Pose loaded
Future_Pose loaded
******************************


In [95]:
net = OO_LSTM_vel(args).to(args.device)
optimizer = optim.Adam(net.parameters(), lr= 0.1)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=8, 
                                                 threshold = 1e-12, verbose=True)
print('='*100)
print('Training ...')

train_s_scores=[]
val_s_scores=[]

for epoch in range(200):
    start = time.time()
    
    avg_epoch_train_s_loss   = 0
    avg_epoch_val_s_loss     = 0 
    ade  = 0
    fde  = 0
    ade_train  = 0
    fde_train  = 0
    counter = 0
    net.train()
    for idx, (obs_s, target_s, obs_pose, target_pose) in enumerate(train_loader):
        counter += 1        
        
        
        obs_pose    = obs_pose.to(device='cuda')
        target_pose = target_pose.to(device='cuda')
        obs_s    = obs_s.to(device='cuda')
        target_s = target_s.to(device='cuda')
 
        net.zero_grad()
    
        (vel_preds,) = net(pose=obs_pose, vel=obs_s)

        loss  = l1e(vel_preds, target_s)
        pose_preds=speed2pos(vel_preds,obs_pose)
    
        ade_train += float(ADE_c(pose_preds, target_pose))
        fde_train += float(FDE_c(pose_preds, target_pose))
        
    
        loss.backward()
        optimizer.step()
        
    
        avg_epoch_train_s_loss += float(loss)

    avg_epoch_train_s_loss /= counter
    train_s_scores.append(avg_epoch_train_s_loss)
    ade_train  /= counter
    fde_train  /= counter   
    
    
  
    counter=0
    net.eval()
    for idx, (obs_s, target_s, obs_pose, target_pose) in enumerate(val_loader):
        counter+=1
       
        obs_pose    = obs_pose.to(device='cuda')
        target_pose = target_pose.to(device='cuda')
        obs_s    = obs_s.to(device='cuda')
        target_s = target_s.to(device='cuda')
        
        with torch.no_grad():
            (vel_preds,) = net(pose=obs_pose, vel=obs_s)

            val_loss  = l1e(vel_preds, target_s)
            pose_preds=speed2pos(vel_preds,obs_pose)
            
            avg_epoch_val_s_loss += float(val_loss)
            
            ade += float(ADE_c(pose_preds, target_pose))
            fde += float(FDE_c(pose_preds, target_pose))

        
    avg_epoch_val_s_loss /= counter
    val_s_scores.append(avg_epoch_val_s_loss)
    
    ade  /= counter
    fde  /= counter     
   
    
    
    scheduler.step(avg_epoch_train_s_loss)
    
    print('e:', epoch,'| loss: %.2f'%avg_epoch_train_s_loss,'| val_loss: %.2f'% avg_epoch_val_s_loss, '| tpose: %.2f'% avg_epoch_train_s_loss, '| vpose: %.2f'% avg_epoch_val_s_loss, '| ade_train: %.2f'% ade_train, '| ade_val: %.2f'% ade, '| fde_train: %.2f'% fde_train,'| fde_val: %.2f'% fde,
          '| t:%.2f'%(time.time()-start))


print('='*100) 
# print('Saving ...')
# torch.save(net.state_dict(), args.model_path)
print('Done !')

Training ...
e: 0 | loss: 5.63 | val_loss: 35.79 | tpose: 5.63 | vpose: 35.79 | ade_train: 83.66 | ade_val: 460.74 | fde_train: 152.59 | fde_val: 836.74 | t:0.49
e: 1 | loss: 14.10 | val_loss: 3.12 | tpose: 14.10 | vpose: 3.12 | ade_train: 181.60 | ade_val: 40.84 | fde_train: 333.20 | fde_val: 77.04 | t:0.49
e: 2 | loss: 2.54 | val_loss: 3.12 | tpose: 2.54 | vpose: 3.12 | ade_train: 31.05 | ade_val: 40.84 | fde_train: 58.64 | fde_val: 77.04 | t:0.48
e: 3 | loss: 2.53 | val_loss: 3.12 | tpose: 2.53 | vpose: 3.12 | ade_train: 30.99 | ade_val: 40.84 | fde_train: 58.49 | fde_val: 77.04 | t:0.47
e: 4 | loss: 2.55 | val_loss: 3.17 | tpose: 2.55 | vpose: 3.17 | ade_train: 31.22 | ade_val: 41.68 | fde_train: 58.95 | fde_val: 77.67 | t:0.48
e: 5 | loss: 2.55 | val_loss: 3.12 | tpose: 2.55 | vpose: 3.12 | ade_train: 31.33 | ade_val: 40.84 | fde_train: 58.84 | fde_val: 77.04 | t:0.49
e: 6 | loss: 2.54 | val_loss: 3.12 | tpose: 2.54 | vpose: 3.12 | ade_train: 31.13 | ade_val: 40.84 | fde_train: 58

e: 55 | loss: 2.53 | val_loss: 3.12 | tpose: 2.53 | vpose: 3.12 | ade_train: 31.00 | ade_val: 40.84 | fde_train: 58.49 | fde_val: 77.04 | t:0.49
e: 56 | loss: 2.55 | val_loss: 3.12 | tpose: 2.55 | vpose: 3.12 | ade_train: 31.19 | ade_val: 40.84 | fde_train: 58.90 | fde_val: 77.04 | t:0.48
e: 57 | loss: 2.55 | val_loss: 3.12 | tpose: 2.55 | vpose: 3.12 | ade_train: 31.27 | ade_val: 40.84 | fde_train: 59.08 | fde_val: 77.04 | t:0.49
e: 58 | loss: 2.54 | val_loss: 3.12 | tpose: 2.54 | vpose: 3.12 | ade_train: 31.07 | ade_val: 40.84 | fde_train: 58.67 | fde_val: 77.04 | t:0.47
e: 59 | loss: 2.54 | val_loss: 3.12 | tpose: 2.54 | vpose: 3.12 | ade_train: 31.08 | ade_val: 40.84 | fde_train: 58.66 | fde_val: 77.04 | t:0.48
e: 60 | loss: 2.54 | val_loss: 3.12 | tpose: 2.54 | vpose: 3.12 | ade_train: 31.14 | ade_val: 40.84 | fde_train: 58.79 | fde_val: 77.04 | t:0.47
e: 61 | loss: 2.55 | val_loss: 3.12 | tpose: 2.55 | vpose: 3.12 | ade_train: 31.29 | ade_val: 40.84 | fde_train: 59.11 | fde_val: 

e: 110 | loss: 2.54 | val_loss: 3.12 | tpose: 2.54 | vpose: 3.12 | ade_train: 31.05 | ade_val: 40.84 | fde_train: 58.61 | fde_val: 77.04 | t:0.47
e: 111 | loss: 2.54 | val_loss: 3.12 | tpose: 2.54 | vpose: 3.12 | ade_train: 31.16 | ade_val: 40.84 | fde_train: 58.83 | fde_val: 77.04 | t:0.48
e: 112 | loss: 2.55 | val_loss: 3.12 | tpose: 2.55 | vpose: 3.12 | ade_train: 31.30 | ade_val: 40.84 | fde_train: 59.15 | fde_val: 77.04 | t:0.48
e: 113 | loss: 2.54 | val_loss: 3.12 | tpose: 2.54 | vpose: 3.12 | ade_train: 31.03 | ade_val: 40.84 | fde_train: 58.58 | fde_val: 77.04 | t:0.47
e: 114 | loss: 2.54 | val_loss: 3.12 | tpose: 2.54 | vpose: 3.12 | ade_train: 31.04 | ade_val: 40.84 | fde_train: 58.57 | fde_val: 77.04 | t:0.47
e: 115 | loss: 2.54 | val_loss: 3.12 | tpose: 2.54 | vpose: 3.12 | ade_train: 31.03 | ade_val: 40.84 | fde_train: 58.59 | fde_val: 77.04 | t:0.48
Epoch   117: reducing learning rate of group 0 to 4.8828e-05.
e: 116 | loss: 2.54 | val_loss: 3.12 | tpose: 2.54 | vpose: 3.

e: 164 | loss: 2.55 | val_loss: 3.12 | tpose: 2.55 | vpose: 3.12 | ade_train: 31.20 | ade_val: 40.84 | fde_train: 58.93 | fde_val: 77.04 | t:0.48
e: 165 | loss: 2.54 | val_loss: 3.12 | tpose: 2.54 | vpose: 3.12 | ade_train: 31.04 | ade_val: 40.84 | fde_train: 58.57 | fde_val: 77.04 | t:0.48
e: 166 | loss: 2.54 | val_loss: 3.12 | tpose: 2.54 | vpose: 3.12 | ade_train: 31.06 | ade_val: 40.84 | fde_train: 58.64 | fde_val: 77.04 | t:0.47
e: 167 | loss: 2.54 | val_loss: 3.12 | tpose: 2.54 | vpose: 3.12 | ade_train: 31.12 | ade_val: 40.84 | fde_train: 58.75 | fde_val: 77.04 | t:0.48
e: 168 | loss: 2.55 | val_loss: 3.12 | tpose: 2.55 | vpose: 3.12 | ade_train: 31.24 | ade_val: 40.84 | fde_train: 59.02 | fde_val: 77.04 | t:0.48
e: 169 | loss: 2.54 | val_loss: 3.12 | tpose: 2.54 | vpose: 3.12 | ade_train: 31.13 | ade_val: 40.84 | fde_train: 58.77 | fde_val: 77.04 | t:0.48
Epoch   171: reducing learning rate of group 0 to 7.6294e-07.
e: 170 | loss: 2.54 | val_loss: 3.12 | tpose: 2.54 | vpose: 3.

In [7]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        # encoder
        self.drop =  nn.Dropout(p=0.5)
        self.enc1= nn.Linear(in_features=34*16, out_features=300)
        self.enc2 = nn.Linear(in_features=300, out_features=300)
#         self.enc3 = nn.Linear(in_features=300, out_features=300)
#         self.enc4 = nn.Linear(in_features=300, out_features=10)
        # decoder 
#         self.dec1 = nn.Linear(in_features=10, out_features=300)
#         self.dec2 = nn.Linear(in_features=75, out_features=150)
        self.dec3 = nn.Linear(in_features=300, out_features=300)
        self.dec4 = nn.Linear(in_features=300, out_features=34*16)
        
    def forward(self, x):
                
        x = self.drop(x)
        x = F.relu(self.enc1(x))
        x = F.relu(self.enc2(x))
#         x = F.relu(self.enc3(x))
#         x = F.relu(self.enc4(x))
#         x = F.relu(self.dec1(x))
#         x = F.relu(self.dec2(x))
#         x = F.relu(self.dec3(x))
        x = F.relu(self.dec4(x))
        return x
    
net = Autoencoder()
# print(net)
net.to('cuda:0')

Autoencoder(
  (drop): Dropout(p=0.5, inplace=False)
  (enc1): Linear(in_features=544, out_features=300, bias=True)
  (enc2): Linear(in_features=300, out_features=300, bias=True)
  (dec3): Linear(in_features=300, out_features=300, bias=True)
  (dec4): Linear(in_features=300, out_features=544, bias=True)
)

In [70]:
optimizer = optim.Adam(net.parameters(), lr= 0.1)
# optimizer = optim.Adadelta(net.parameters(),lr= 0.1)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=10, 
                                                 threshold = 1e-12, verbose=True)


train_p_scores=[]
val_p_scores=[]


for epoch in range(300):
    start = time.time()
    avg_epoch_train_p_loss   = 0
    avg_epoch_val_p_loss     = 0 
    ade  = 0
    fde  = 0
    ade_train  = 0
    fde_train  = 0
    counter = 0
    net.train()
    for idx, (obs_s, target_s, obs_pose, target_pose) in enumerate(train_loader):
        counter += 1        
        
        obs_pose    = obs_s.to(device='cuda')
        target_pose = obs_s.to(device='cuda')
#         print(obs_pose.shape)
        obs_pose=obs_pose.reshape(-1,16*34)
        target_pose=target_pose.reshape(-1,16*34)

        net.zero_grad()
    
        pose_preds = net(obs_pose)

        loss  = l1e(pose_preds, target_pose)
    
        ade_train += float(ADE_c(pose_preds.reshape(-1,16,34), target_pose.reshape(-1,15,34)))
        
        loss.backward()
        optimizer.step()
    
        avg_epoch_train_p_loss += float(loss)

    avg_epoch_train_p_loss /= counter
    train_p_scores.append(avg_epoch_train_p_loss)
    ade_train  /= counter
    
    counter=0
    net.eval()
    for idx, (obs_s, target_s, obs_pose, target_pose) in enumerate(val_loader):
        counter+=1
       
        obs_pose    = obs_s.to(device='cuda')
        target_pose = obs_s.to(device='cuda')
        
        obs_pose=obs_pose.reshape(-1,16*34)
        target_pose=target_pose.reshape(-1,16*34)
        
        with torch.no_grad():
            pose_preds = net(obs_pose)
            
            val_loss  = l1e(pose_preds, target_pose)
            avg_epoch_val_p_loss += float(val_loss)
            
            ade += float(ADE_c(pose_preds.reshape(-1,16,34), target_pose.reshape(-1,15,34)))
            
    avg_epoch_val_p_loss /= counter
    val_p_scores.append(avg_epoch_val_p_loss)
    
    ade  /= counter
    
    scheduler.step(avg_epoch_val_p_loss)
    
    print('e:', epoch,'| loss: %.2f'%avg_epoch_train_p_loss,'| val_loss: %.2f'% avg_epoch_val_p_loss, '| ade_train: %.2f'% ade_train, '| ade_val: %.2f'% ade,
          '| t:%.2f'%(time.time()-start))


print('='*100) 
# print('Saving ...')
# torch.save(net.state_dict(), args.model_path)
print('Done !')

RuntimeError: shape '[-1, 544]' is invalid for input of size 25500

In [15]:
class LSTM_spat(nn.Module):
    def __init__(self, args):
        '''
           input: observed body poses and velocites 
           output: predicted poses(not trained), velocities of poses, and intentions
        '''
        super(LSTM_spat, self).__init__()
#         self.encoded_tot=16*20
        self.encoded_size=20
         
        self.pose_encoder = nn.LSTM(input_size=self.encoded_size, hidden_size=args.hidden_size)        
        self.pose_embedding = nn.Sequential(nn.Linear(in_features=args.hidden_size, out_features=self.encoded_size),
                                           nn.ReLU())
        
        self.pose_decoder = nn.LSTMCell(input_size=self.encoded_size, hidden_size=args.hidden_size)
        self.fc_pose   = nn.Linear(in_features=args.hidden_size, out_features=self.encoded_size)
        
        self.hardtanh = nn.Hardtanh(min_val=-1*args.hardtanh_limit,max_val=args.hardtanh_limit)
        self.relu = nn.ReLU() 
        self.softmax = nn.Softmax(dim=1)
        
        self.enc1= nn.Linear(in_features=34, out_features=30)
        self.enc2 = nn.Linear(in_features=30, out_features=self.encoded_size)
        self.dec = nn.Linear(in_features=self.encoded_size, out_features=34)
        
#         self.drop=n
        
#         self.encoder = lstm_encoder(input_size = 34, hidden_size = 1000, num_layers = 3)
#         self.decoder = lstm_decoder(input_size = 34, hidden_size = 1000, num_layers = 3)

#         self.teacher_forcing_ratio = 0.6
        
        self.args = args
        
    def forward(self, pose=None, vel=None, target_pose=None):
        
        poses=pose.permute(1,0,2)

        pose_encoded = torch.tensor([], device=self.args.device)
        pose_recs = torch.tensor([], device=self.args.device)
        
        for i in range(pose.size()[0]):
            x = self.relu(self.enc1(pose[i]))
            x = self.relu(self.enc2(x))
            recreated_pose=self.relu(self.dec(x))
            pose_encoded = torch.cat((pose_encoded, x.unsqueeze(1)), dim = 1)
            pose_recs = torch.cat((pose_recs, recreated_pose.unsqueeze(1)), dim = 1)
        
        
        
        _, (hidden_pose, cell_pose) = self.pose_encoder(pose_encoded)
        hidden_pose = hidden_pose.squeeze(0)
        cell_pose = cell_pose.squeeze(0)
        
        outputs = []
        pose_outputs   = torch.tensor([], device=self.args.device)
        PoseDec_inp = pose_encoded.permute(1,0,2)[:,-1,:]
        
#         print(PoseDec_inp.shape)
        hidden_dec=hidden_pose
        cell_dec=cell_pose
#         print(hidden_dec.shape)
        for i in range(self.args.output//self.args.skip):
        
            hidden_dec, cell_dec = self.pose_decoder(PoseDec_inp, (hidden_dec, cell_dec))
            pose_output_encoded  = self.fc_pose(hidden_dec)
            
            pose_output= self.relu(self.dec(pose_output_encoded))
            pose_outputs = torch.cat((pose_outputs, pose_output.unsqueeze(1)), dim = 1)
            PoseDec_inp  = pose_output_encoded.detach()
            
        
        pose_recs= pose_recs.permute(1,0,2)
        
#         print(recreated_pose.shape)
#         print(pose_outputs.shape)
#         print(pose_outputs.shape,pose_recs.shape)
        outputs.append(pose_outputs)
        outputs.append(pose_recs)

            
        return tuple(outputs)

net = LSTM_spat(args).to(args.device)

In [16]:

optimizer = optim.Adam(net.parameters(), lr= 0.1)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=8, 
                                                 threshold = 1e-12, verbose=True)
print('='*100)
print('Training ...')

train_p_scores=[]
val_p_scores=[]

for epoch in range(100):
    start = time.time()
    
    avg_epoch_train_p_loss   = 0
    avg_epoch_val_p_loss     = 0 
    ade  = 0
    fde  = 0
    ade_train  = 0
    fde_train  = 0
    counter = 0
    net.train()
    for idx, (obs_s, target_s, obs_pose, target_pose) in enumerate(train_loader):
        counter += 1        
        
        
        obs_pose    = obs_pose.to(device='cuda')
        target_pose = target_pose.to(device='cuda')
        
        obs_s    = obs_s.to(device='cuda')
        target_s = target_s.to(device='cuda')
 
        net.zero_grad()
    
        (pose_preds,pose_recs) = net(pose=obs_pose, target_pose=target_pose,vel=obs_s)

        loss  = l1e(pose_preds, target_pose)#+l1e(pose_recs, obs_pose)
    
        ade_train += float(ADE_c(pose_preds, target_pose))
        fde_train += float(FDE_c(pose_preds, target_pose))
        
    
        loss.backward()
        optimizer.step()
        
    
        avg_epoch_train_p_loss += float(loss)

    avg_epoch_train_p_loss /= counter
    train_p_scores.append(avg_epoch_train_p_loss)
    ade_train  /= counter
    fde_train  /= counter   
    
    
  
    counter=0
    net.eval()
    for idx, (obs_s, target_s, obs_pose, target_pose) in enumerate(val_loader):
        counter+=1
       
        obs_pose    = obs_pose.to(device='cuda')
        target_pose = target_pose.to(device='cuda')
        
        with torch.no_grad():
            (pose_preds,pose_recs) = net(pose=obs_pose, target_pose=target_pose)
          
            val_loss  = l1e(pose_preds, target_pose)#+l1e(pose_recs, obs_pose)
            avg_epoch_val_p_loss += float(val_loss)
            
            ade += float(ADE_c(pose_preds, target_pose))
            fde += float(FDE_c(pose_preds, target_pose))

        
    avg_epoch_val_p_loss /= counter
    val_p_scores.append(avg_epoch_val_p_loss)
    
    ade  /= counter
    fde  /= counter     
   
    
    
    scheduler.step(avg_epoch_train_p_loss)
    
    print('e:', epoch,'| loss: %.2f'%avg_epoch_train_p_loss,'| val_loss: %.2f'% avg_epoch_val_p_loss, '| tpose: %.2f'% avg_epoch_train_p_loss, '| vpose: %.2f'% avg_epoch_val_p_loss, '| ade_train: %.2f'% ade_train, '| ade_val: %.2f'% ade, '| fde_train: %.2f'% fde_train,'| fde_val: %.2f'% fde,
          '| t:%.2f'%(time.time()-start))


print('='*100) 
# print('Saving ...')
# torch.save(net.state_dict(), args.model_path)
print('Done !')

Training ...
e: 0 | loss: 825.09 | val_loss: 696.44 | tpose: 825.09 | vpose: 696.44 | ade_train: 1222.62 | ade_val: 1068.66 | fde_train: 1215.08 | fde_val: 1051.94 | t:0.36
e: 1 | loss: 510.10 | val_loss: 397.66 | tpose: 510.10 | vpose: 397.66 | ade_train: 801.87 | ade_val: 611.26 | fde_train: 793.59 | fde_val: 617.65 | t:0.35
e: 2 | loss: 434.34 | val_loss: 328.93 | tpose: 434.34 | vpose: 328.93 | ade_train: 669.38 | ade_val: 533.41 | fde_train: 694.11 | fde_val: 535.86 | t:0.34
e: 3 | loss: 351.86 | val_loss: 392.16 | tpose: 351.86 | vpose: 392.16 | ade_train: 565.53 | ade_val: 615.04 | fde_train: 558.21 | fde_val: 594.49 | t:0.34
e: 4 | loss: 340.28 | val_loss: 324.46 | tpose: 340.28 | vpose: 324.46 | ade_train: 545.77 | ade_val: 525.71 | fde_train: 530.88 | fde_val: 529.32 | t:0.34
e: 5 | loss: 321.77 | val_loss: 297.82 | tpose: 321.77 | vpose: 297.82 | ade_train: 525.19 | ade_val: 507.84 | fde_train: 536.98 | fde_val: 520.92 | t:0.33
e: 6 | loss: 301.73 | val_loss: 310.05 | tpose:

e: 52 | loss: 275.30 | val_loss: 284.51 | tpose: 275.30 | vpose: 284.51 | ade_train: 478.77 | ade_val: 492.01 | fde_train: 491.33 | fde_val: 511.01 | t:0.33
e: 53 | loss: 276.21 | val_loss: 286.03 | tpose: 276.21 | vpose: 286.03 | ade_train: 479.34 | ade_val: 494.54 | fde_train: 491.59 | fde_val: 513.64 | t:0.35
e: 54 | loss: 276.71 | val_loss: 288.72 | tpose: 276.71 | vpose: 288.72 | ade_train: 481.27 | ade_val: 500.53 | fde_train: 493.18 | fde_val: 519.92 | t:0.33
e: 55 | loss: 274.81 | val_loss: 284.92 | tpose: 274.81 | vpose: 284.92 | ade_train: 477.68 | ade_val: 493.12 | fde_train: 489.64 | fde_val: 512.12 | t:0.33
e: 56 | loss: 275.46 | val_loss: 283.21 | tpose: 275.46 | vpose: 283.21 | ade_train: 478.60 | ade_val: 489.14 | fde_train: 491.44 | fde_val: 508.41 | t:0.32
Epoch    58: reducing learning rate of group 0 to 2.5000e-02.
e: 57 | loss: 275.51 | val_loss: 281.96 | tpose: 275.51 | vpose: 281.96 | ade_train: 477.87 | ade_val: 487.63 | fde_train: 490.83 | fde_val: 507.00 | t:0

In [4]:
class myDataset_DE(torch.utils.data.Dataset):
    def __init__(self, args, dtype):
        
        self.args = args
        self.dtype = dtype
        print("Loading",self.dtype)
        
        sequence_centric = pd.read_csv("sequences_openpifpaf_"+self.dtype+".csv")
#         sequence_centric = pd.read_csv("sequences_4_"+self.dtype+".csv")

        df = sequence_centric.copy()      
        for v in list(df.columns.values):
            print(v+' loaded')
            try:
                df.loc[:,v] = df.loc[:, v].apply(lambda x: literal_eval(x))
            except:
                continue
        sequence_centric[df.columns] = df[df.columns]
        self.data = sequence_centric.copy().reset_index(drop=True)
    
        print('*'*30)
        

    def __len__(self):
        return len(self.data)
    
    
    def __getitem__(self, index):

        seq = self.data.iloc[index]
        outputs = []

        obs = torch.tensor([seq.Pose[i] for i in range(0,self.args.input,self.args.skip)])        
        obs_speed = (obs[1:] - obs[:-1])
        
        true = torch.tensor([seq.Future_Pose[i] for i in range(0,self.args.output,self.args.skip)])
        true_speed = torch.cat(((true[0]-obs[-1]).unsqueeze(0), true[1:]-true[:-1]))
        
        outputs.append(obs_speed)
        outputs.append(true_speed)
        #obs and future global avg of joint 5 & 6 (0 indexed) for openpifpaf
        #local = obs pose-global  for openpifpaf
        #do serparately for x and y
        obs_resh = torch.reshape(obs, (obs.size()[0],17,2))
        
        obs_global=(obs_resh[:,5]+obs_resh[:,6])/2.0
        obs_global=obs_global.unsqueeze(1)

        obs_resh=obs_resh-obs_global
    
#         print("-"*100,"\n",obs)
#         print("-"*100,"\n",obs_resh)
#         print("-"*100,"\n",obs_global)
#         print("-"*100,"\n",obs_resh+obs_global)
    
        obs_resh=obs_resh.reshape(obs.size())
        
        true_resh = torch.reshape(true, (true.size()[0],17,2))
        true_global=(true_resh[:,5]+true_resh[:,6])/2.0
        true_global=true_global.unsqueeze(1)
#         print(obs_resh.size(),obs_global.size())
        true_resh=true_resh-true_global
        true_resh=true_resh.reshape(true.size())
        
        obs_global=torch.reshape(obs_global, (obs.size()[0],2))
        true_global=torch.reshape(true_global, (true.size()[0],2))
        
        obs_global_speed = (obs_global[1:] - obs_global[:-1])
        true_global_speed = torch.cat(((true_global[0]-obs_global[-1]).unsqueeze(0), true_global[1:]-true_global[:-1]))
        
#         print(obs_resh.size(),true_resh.size(),obs_global.size(),true_global.size())
        
        outputs.append(obs_resh)
        outputs.append(true_resh)
        outputs.append(obs_global)
        outputs.append(true_global)
        outputs.append(obs_global_speed)
        outputs.append(true_global_speed)
        outputs.append(true)
#         print(obs_resh.size())
#         obs_y_global=(obs_resh[:,5,1]+obs_resh[:,6,1])/2.0
#         print(obs_global.size())
#         print(obs_global)
#         print(obs_y_global)
        
#         print(obs.size(),obs_resh.size())
#         obs_global= torch.mean(obs[:,5:7],dim=1).unsqueeze(1)
#         print(obs_global)
#         obs=obs-obs_global
#         print(obs)
#         true_global=torch.mean(true[:,5:7],dim=1).unsqueeze(1)
#         true=true-true_global
       
        
        
        return tuple(outputs)    
    
    
def data_loader_DE(args,data):
    dataset = myDataset_DE(args,data)
    dataloader = torch.utils.data.DataLoader(
        dataset, batch_size=args.batch_size, shuffle=args.loader_shuffle,
        pin_memory=args.pin_memory)

    return dataloader
   

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import torchvision
import torchvision.transforms as transforms


class PV_LSTM_DE_op(nn.Module):
    def __init__(self, args):
        '''
           input: observed body poses and velocites global and local
           output: global and local velocities
        '''
        super(PV_LSTM_DE_op, self).__init__()
         
        self.pose_encoder = nn.LSTM(input_size=34, hidden_size=args.hidden_size)
        self.vel_encoder = nn.LSTM(input_size=34, hidden_size=args.hidden_size)
        self.vel_decoder = nn.LSTMCell(input_size=34, hidden_size=args.hidden_size)
        self.fc_vel    = nn.Linear(in_features=args.hidden_size, out_features=34)
        
        self.pose_glob_encoder = nn.LSTM(input_size=2, hidden_size=100)
        self.vel_glob_encoder = nn.LSTM(input_size=2, hidden_size=100)
        self.vel_glob_decoder = nn.LSTMCell(input_size=2, hidden_size=100)
        self.fc_global    = nn.Linear(in_features=100, out_features=2)
        
        
        
        self.hardtanh = nn.Hardtanh(min_val=-1*args.hardtanh_limit,max_val=args.hardtanh_limit)
        self.relu = nn.ReLU() 
        self.softmax = nn.Softmax(dim=1)
        
        
        self.args = args
        
    def forward(self, pose=None, vel=None, pose_glob=None, vel_glob=None):


        _, (hidden_vel, cell_vel) = self.vel_encoder(vel.permute(1,0,2))
        hidden_vel = hidden_vel.squeeze(0)
        cell_vel = cell_vel.squeeze(0)


        _, (hidden_pose, cell_pose) = self.pose_encoder(pose.permute(1,0,2))
        hidden_pose = hidden_pose.squeeze(0)
        cell_pose = cell_pose.squeeze(0)
        
        outputs = []
        
      
        #predicting the velocity of poses and poses
        vel_outputs    = torch.tensor([], device=self.args.device)
        #pose_outputs   = torch.tensor([], device=self.args.device)
        glob_outputs    = torch.tensor([], device=self.args.device)

        VelDec_inp = vel[:,-1,:]
        #PoseDec_inp = pose[:,-1,:]
        
        hidden_dec = hidden_pose + hidden_vel
        cell_dec = cell_pose + cell_vel
        for i in range(self.args.output//self.args.skip):
            hidden_dec, cell_dec = self.vel_decoder(VelDec_inp, (hidden_dec, cell_dec))
            vel_output  = self.hardtanh(self.fc_vel(hidden_dec))
            vel_outputs = torch.cat((vel_outputs, vel_output.unsqueeze(1)), dim = 1)
            VelDec_inp  = vel_output.detach()
          
        outputs.append(vel_outputs)
        
        _, (hidden_vel_glob, cell_vel_glob) = self.vel_glob_encoder(vel_glob.permute(1,0,2))
        hidden_vel_glob = hidden_vel_glob.squeeze(0)
        cell_vel_glob = cell_vel_glob.squeeze(0)
        
        _, (hidden_pose_glob, cell_pose_glob) = self.pose_glob_encoder(pose_glob.permute(1,0,2))
        hidden_pose_glob = hidden_pose_glob.squeeze(0)
        cell_pose_glob = cell_pose_glob.squeeze(0)
        
        glob_inp = pose_glob[:,-1,:]
        
        hidden_glob = hidden_pose_glob + hidden_vel_glob
        cell_glob = cell_pose_glob + cell_vel_glob
        for i in range(self.args.output//self.args.skip):
            hidden_glob, cell_glob = self.vel_glob_decoder(glob_inp, (hidden_glob, cell_glob))
            glob_output  = self.fc_global(hidden_glob)
            glob_outputs = torch.cat((glob_outputs, glob_output.unsqueeze(1)), dim = 1)
            glob_inp  = glob_output.detach()
       
        outputs.append(glob_outputs)
        
       
        return tuple(outputs)


class myDataset_DE_op(torch.utils.data.Dataset):
    def __init__(self, args, dtype):
        
        self.args = args
        self.dtype = dtype
        print("Loading",self.dtype)
        
        sequence_centric = pd.read_csv("sequences_openpifpaf_thres4_wimage_"+self.dtype+".csv")
#         sequences_openpifpaf_thres4_wconfsc_wimage_train
#         sequence_centric = pd.read_csv("sequences_16_overlap_4_thres4_"+self.dtype+".csv")

        df = sequence_centric.copy()      
        for v in list(df.columns.values):
            print(v+' loaded')
            try:
                df.loc[:,v] = df.loc[:, v].apply(lambda x: literal_eval(x))
            except:
                continue
        sequence_centric[df.columns] = df[df.columns]
        self.data = sequence_centric.copy().reset_index(drop=True)
    
        print('*'*30)
        

    def __len__(self):
        return len(self.data)
    
    
    def __getitem__(self, index):

        seq = self.data.iloc[index]
        outputs = []

        obs = torch.tensor([seq.Pose[i] for i in range(0,self.args.input,self.args.skip)])        
        obs_speed = (obs[1:] - obs[:-1])
        
        true = torch.tensor([seq.Future_Pose[i] for i in range(0,self.args.output,self.args.skip)])
        true_speed = torch.cat(((true[0]-obs[-1]).unsqueeze(0), true[1:]-true[:-1]))
        
        outputs.append(obs_speed)
        outputs.append(true_speed)
        #obs and future global avg of joint 5 & 6 (0 indexed) for openpifpaf
        #local = obs pose-global  for openpifpaf
        #do serparately for x and y
        obs_resh = torch.reshape(obs, (obs.size()[0],17,2))
        
        obs_global=obs_resh[:,0]
        obs_global=obs_global.unsqueeze(1)

        obs_resh=obs_resh-obs_global
    

        obs_resh=obs_resh.reshape(obs.size())
        
        true_resh = torch.reshape(true, (true.size()[0],17,2))
        true_global=true_resh[:,0]
        true_global=true_global.unsqueeze(1)
#         print(obs_resh.size(),obs_global.size())
        true_resh=true_resh-true_global
        true_resh=true_resh.reshape(true.size())
        
        obs_global=torch.reshape(obs_global, (obs.size()[0],2))
        true_global=torch.reshape(true_global, (true.size()[0],2))
        
        obs_global_speed = (obs_global[1:] - obs_global[:-1])
        true_global_speed = torch.cat(((true_global[0]-obs_global[-1]).unsqueeze(0), true_global[1:]-true_global[:-1]))

        outputs.append(obs_resh)
        outputs.append(true_resh)
        outputs.append(obs_global)
        outputs.append(true_global)
        outputs.append(obs_global_speed)
        outputs.append(true_global_speed)
        outputs.append(true)

        return tuple(outputs)    
    
    
def data_loader_DE_op(args,data):
    dataset = myDataset_DE_op(args,data)
    dataloader = torch.utils.data.DataLoader(
        dataset, batch_size=args.batch_size, shuffle=args.loader_shuffle,
        pin_memory=args.pin_memory)

    return dataloader

#Interpolation

In [240]:
torch.autograd.set_detect_anomaly(True)
class glob_LSTM_vel(nn.Module):    
    def __init__(self, args):
        '''
           input: observed body poses and velocites 
           output: predicted poses(not trained), velocities of poses, and intentions
        '''
        super(glob_LSTM_vel, self).__init__()
         
        self.pose_encoder_g = nn.LSTM(input_size=2, hidden_size=50)
        self.vel_encoder_g = nn.LSTM(input_size=2, hidden_size=50)
        self.vel_decoder_g = nn.LSTMCell(input_size=2, hidden_size=50)
        self.fc_vel_g    = nn.Linear(in_features=50, out_features=2)
        
        self.pose_encoder = nn.LSTM(input_size=34, hidden_size=1000,num_layers=3)
        self.vel_encoder = nn.LSTM(input_size=34, hidden_size=1000,num_layers=3)
        self.vel_decoder = nn.LSTM(input_size=34, hidden_size=1000,num_layers=3)
        self.fc_vel    = nn.Linear(in_features=1000, out_features=34)
        
        self.hardtanh = nn.Hardtanh(min_val=-1*args.hardtanh_limit,max_val=args.hardtanh_limit)
        self.relu = nn.ReLU() 
        self.softmax = nn.Softmax(dim=1)
        
        self.args = args
        
    def forward(self, pose=None, vel=None, pose_g=None,vel_g=None):
        
        outputs = []        
#         _, (hidden_s,cell_s) = self.vel_encoder(vel.permute(1,0,2))             
#         _, (hidden_p,cell_p) = self.pose_encoder(pose.permute(1,0,2))
#         hidden_dec = hidden_s + hidden_p
#         cell_dec = cell_s + cell_p

#         VelDec_inp = vel[:,-1,:].unsqueeze(0)
       
        vel_outputs = torch.tensor([], device=self.args.device)

#         print(VelDec_inp.shape)
#         for i in range(self.args.output//self.args.skip):
# #             print(hidden_dec[0].shape,hidden_dec[1].shape)
#             decoder_output, (hidden_dec,cell_dec) = self.vel_decoder(VelDec_inp, (hidden_dec,cell_dec))
#             vel_output  = self.fc_vel(decoder_output)
# #             print(vel_output.shape)
#             vel_outputs = torch.cat((vel_outputs, vel_output), dim = 0)
#             VelDec_inp  = vel_output.detach()
            
            
        outputs.append(vel_outputs)#.permute(1,0,2)
   
        
        _, (hidden_vel_g, cell_vel_g) = self.vel_encoder_g(vel_g.permute(1,0,2))
        hidden_vel_g = hidden_vel_g.squeeze(0)
        cell_vel_g = cell_vel_g.squeeze(0)

        _, (hidden_pose_g, cell_pose_g) = self.pose_encoder_g(pose_g.permute(1,0,2))
        hidden_pose_g = hidden_pose_g.squeeze(0)
        cell_pose_g = cell_pose_g.squeeze(0)

                
        vel_outputs_g    = torch.tensor([], device=self.args.device)

        VelDec_inp_g = vel_g[:,-1,:]
        hidden_dec_g = hidden_pose_g + hidden_vel_g
        cell_dec_g = cell_pose_g + cell_vel_g
#         print(VelDec_inp_g.shape)
        
        for i in range(self.args.output//self.args.skip):
            hidden_dec_g, cell_dec_g = self.vel_decoder_g(VelDec_inp_g, (hidden_dec_g, cell_dec_g))
            vel_output_g  = self.fc_vel_g(hidden_dec_g)
            vel_outputs_g = torch.cat((vel_outputs_g, vel_output_g.unsqueeze(1)), dim = 1)
            VelDec_inp_g  = vel_output_g.detach()
            
        outputs.append(vel_outputs_g)


            
        return tuple(outputs)

def speed2pos_glob(preds, glob_pred, obs_p, obs_global) :
    pred_pos = torch.zeros(preds.shape[0], preds.shape[1], 34).to('cuda')

    current = (obs_p[:,-1,:].reshape(-1,17,2)+ obs_global[:,-1,:].reshape(-1,1,2)).reshape(-1,34)
    
    for i in range(preds.shape[1]):
        pred_pos[:,i,:] = ((current + preds[:,i,:]).reshape(current.size()[0],17,2)+glob_pred[:,i,:].reshape(-1,1,2)).reshape(-1,34)
        current = pred_pos[:,i,:]
        
    for i in range(34):
        pred_pos[:,:,i] = torch.min(pred_pos[:,:,i], 1920*torch.ones(pred_pos.shape[0], pred_pos.shape[1], device='cuda'))
        pred_pos[:,:,i] = torch.max(pred_pos[:,:,i], torch.zeros(pred_pos.shape[0], pred_pos.shape[1], device='cuda'))
        
    return pred_pos

In [ ]:

class args():
    def __init__(self):
        self.jaad_dataset = '/data/smailait-data/JAAD/processed_annotations' #folder containing parsed jaad annotations (used when first time loading data)
        self.dtype        = 'train'
        self.from_file    = False #read dataset from csv file or reprocess data
        self.save         = True
        self.file         = '/data/smailait-data/jaad_train_16_16.csv'
        self.save_path    = '/data/smailait-data/jaad_train_16_16.csv'
        self.model_path    = '/data/smailait-data/models/multitask_pv_lstm_trained.pkl'
        self.loader_workers = 1
        self.loader_shuffle = True
        self.pin_memory     = False
        self.image_resize   = [240, 426]
        self.device         = 'cuda'
        self.batch_size     = 50
        self.n_epochs       = 250
        self.hidden_size    = 1000
        self.hardtanh_limit = 100
        self.input  = 16
        self.output = 16
        self.stride = 16
        self.skip   = 1
        # self.task   = 'bounding_box-intention'
        self.task   = 'pose'
        self.use_scenes = False       
        self.lr = 0.01
        
args = args()

# net = PV_LSTM(args).to(args.device)
net = glob_LSTM_vel(args).to(args.device)#PV_LSTM_DE_op(args).to(args.device)
# train = DataLoader.data_loader(args)
# args.dtype = 'val'
# args.save_path = args.save_path.replace('train', 'val')
# args.file = args.file.replace('train', 'val')
# val = DataLoader.data_loader(args)

# optimizer = optim.Adam(net.parameters(), lr=args.lr)
# scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=15, 
#                                                  threshold = 1e-8, verbose=True)
mse = nn.MSELoss()
l1e = nn.L1Loss()
bce = nn.BCELoss()
train_s_scores = []
train_pose_scores=[]
val_pose_scores=[]
train_c_scores = []
val_s_scores   = []
val_c_scores   = []
train_g_scores = []
val_g_scores   = []

train_loader=data_loader_DE_op(args,"train")
val_loader=data_loader_DE_op(args,"val" )

optimizer = optim.Adam(net.parameters(), lr= 0.1)
# optimizer = optim.Adadelta(net.parameters(),lr= 0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=8, 
                                                 threshold = 1e-12, verbose=True)



In [244]:
print('='*100)
print('Training ...')

train_p_scores=[]
val_p_scores=[]
alpha=1#0.4

for epoch in range(300):
    start = time.time()
    
    avg_epoch_train_p_loss   = 0
    avg_epoch_val_p_loss     = 0 
    ade  = 0
    fde  = 0
    ade_train  = 0
    fde_train  = 0
    ade_g  = 0
    fde_g  = 0
    ade_train_g  = 0
    fde_train_g  = 0
    counter = 0
    net.train()
    
    for idx, (obs_s, target_s, obs_pose, target_pose, obs_pose_global, target_pose_global, obs_s_g, target_s_g, target) in enumerate(train_loader):
#         print("SAMPLE",idx,"."*50)
#         obs_s = obs_s.to(device='cuda')
#         target_s = target_s.to(device='cuda')
#         obs_pose    = obs_pose.to(device='cuda')
#         target_pose = target_pose.to(device='cuda')
        obs_pose_global = obs_pose_global.to(device='cuda')
        target_pose_global = target_pose_global.to(device='cuda')
        obs_s_g  = obs_s_g.to(device='cuda')
        target_s_g = target_s_g.to(device='cuda')
#         target = target.to(device='cuda')   
#         print(obs_pose_global.shape)
#         print(obs_s_g.shape)
        counter += 1        
#         break
        
        
        net.zero_grad()
    
        (preds_s,preds_s_g) = net(pose_g=obs_pose_global, vel_g=obs_s_g,pose=obs_pose, vel=obs_s)
        
#         print(preds_s.shape,preds_s_g.shape)
#         pose_preds=speed2pos(preds_s,obs_pose)
        pose_preds_g=speed2pos(preds_s_g,obs_pose_global)
    
#         ade_train += float(ADE_c(pose_preds, target_pose))
#         fde_train += float(FDE_c(pose_preds, target_pose))
        ade_train_g += float(ADE_c(pose_preds_g, target_pose_global))
        fde_train_g += float(FDE_c(pose_preds_g, target_pose_global))
        
        #speed2pos_glob(preds, glob_pred, obs_p, obs_global)
        
#         loss_l  = l1e(preds_s, target_s)
        loss_g  = l1e(preds_s_g, target_s_g)
        
        loss=alpha*loss_g# + (1-alpha)*loss_l
        

    
        loss.backward()
        optimizer.step()
        
    
        avg_epoch_train_p_loss += float(loss)

    avg_epoch_train_p_loss /= counter
    train_p_scores.append(avg_epoch_train_p_loss)
    ade_train  /= counter
    fde_train  /= counter   
    ade_train_g  /= counter
    fde_train_g  /= counter   
    
#     break
  
    counter=0
    net.eval()
    for idx, (obs_s, target_s, obs_pose, target_pose, obs_pose_global, target_pose_global, obs_s_g, target_s_g, target) in enumerate(val_loader):
#         print("SAMPLE",idx,"."*50)
#         obs_s = obs_s.to(device='cuda')
#         target_s = target_s.to(device='cuda')
#         obs_pose    = obs_pose.to(device='cuda')
#         target_pose = target_pose.to(device='cuda')
        obs_pose_global = obs_pose_global.to(device='cuda')
        target_pose_global = target_pose_global.to(device='cuda')
        obs_s_g  = obs_s_g.to(device='cuda')
        target_s_g = target_s_g.to(device='cuda')
#         target = target.to(device='cuda')   
        
        counter += 1        

        
        with torch.no_grad():
            (preds_s,preds_s_g) = net(pose_g=obs_pose_global, vel_g=obs_s_g,pose=obs_pose, vel=obs_s)

#             loss_l  = l1e(preds_s, target_s)
            loss_g  = l1e(preds_s_g, target_s_g)
        
            val_loss=alpha*loss_g #+ (1-alpha)*loss_l
            
            avg_epoch_val_p_loss += float(val_loss)
            
            
#             pose_preds=speed2pos(preds_s,obs_pose)
            pose_preds_g=speed2pos(preds_s_g,obs_pose_global)
            
#             ade += float(ADE_c(pose_preds, target_pose))
#             fde += float(FDE_c(pose_preds, target_pose))
            ade_g += float(ADE_c(pose_preds_g, target_pose_global))
            fde_g += float(FDE_c(pose_preds_g, target_pose_global))
        
        
    avg_epoch_val_p_loss /= counter
    val_p_scores.append(avg_epoch_val_p_loss)
    
    ade  /= counter
    fde  /= counter    
    ade_g  /= counter
    fde_g  /= counter    
   
    
    
    scheduler.step(avg_epoch_train_p_loss)
    
    print('e:', epoch,'| loss: %.2f'%avg_epoch_train_p_loss,'| val_loss: %.2f'% avg_epoch_val_p_loss, '| ade_train_l: %.2f'% ade_train, '| ade_val_l: %.2f'% ade, '| fde_train_l: %.2f'% fde_train,'| fde_val_l: %.2f'% fde, '| ade_train_g: %.2f'% ade_train_g, '| ade_val_g: %.2f'% ade_g, '| fde_train_g: %.2f'% fde_train_g,'| fde_val_g: %.2f'% fde_g)


print('='*100) 
# print('Saving ...')
# torch.save(net.state_dict(), args.model_path)
print('Done !')

Training ...
e: 0 | loss: 2.59 | val_loss: 4.32 | ade_train_l: 0.00 | ade_val_l: 0.00 | fde_train_l: 0.00 | fde_val_l: 0.00 | ade_train_g: 33.28 | ade_val_g: 28.59 | fde_train_g: 65.03 | fde_val_g: 54.31
e: 1 | loss: 2.16 | val_loss: 4.54 | ade_train_l: 0.00 | ade_val_l: 0.00 | fde_train_l: 0.00 | fde_val_l: 0.00 | ade_train_g: 25.62 | ade_val_g: 24.37 | fde_train_g: 50.81 | fde_val_g: 44.78
e: 2 | loss: 2.03 | val_loss: 3.88 | ade_train_l: 0.00 | ade_val_l: 0.00 | fde_train_l: 0.00 | fde_val_l: 0.00 | ade_train_g: 23.54 | ade_val_g: 21.64 | fde_train_g: 46.90 | fde_val_g: 40.32
e: 3 | loss: 1.75 | val_loss: 3.24 | ade_train_l: 0.00 | ade_val_l: 0.00 | fde_train_l: 0.00 | fde_val_l: 0.00 | ade_train_g: 19.10 | ade_val_g: 18.23 | fde_train_g: 37.97 | fde_val_g: 35.17
e: 4 | loss: 1.71 | val_loss: 3.66 | ade_train_l: 0.00 | ade_val_l: 0.00 | fde_train_l: 0.00 | fde_val_l: 0.00 | ade_train_g: 18.46 | ade_val_g: 17.35 | fde_train_g: 37.04 | fde_val_g: 32.71
e: 5 | loss: 1.77 | val_loss: 3.

e: 42 | loss: 1.41 | val_loss: 2.93 | ade_train_l: 0.00 | ade_val_l: 0.00 | fde_train_l: 0.00 | fde_val_l: 0.00 | ade_train_g: 13.35 | ade_val_g: 12.89 | fde_train_g: 27.01 | fde_val_g: 24.91
e: 43 | loss: 1.41 | val_loss: 2.98 | ade_train_l: 0.00 | ade_val_l: 0.00 | fde_train_l: 0.00 | fde_val_l: 0.00 | ade_train_g: 13.39 | ade_val_g: 13.55 | fde_train_g: 26.83 | fde_val_g: 26.25
e: 44 | loss: 1.42 | val_loss: 3.43 | ade_train_l: 0.00 | ade_val_l: 0.00 | fde_train_l: 0.00 | fde_val_l: 0.00 | ade_train_g: 13.48 | ade_val_g: 13.49 | fde_train_g: 27.26 | fde_val_g: 25.07
e: 45 | loss: 1.39 | val_loss: 2.92 | ade_train_l: 0.00 | ade_val_l: 0.00 | fde_train_l: 0.00 | fde_val_l: 0.00 | ade_train_g: 13.24 | ade_val_g: 13.20 | fde_train_g: 26.74 | fde_val_g: 24.98
e: 46 | loss: 1.40 | val_loss: 2.89 | ade_train_l: 0.00 | ade_val_l: 0.00 | fde_train_l: 0.00 | fde_val_l: 0.00 | ade_train_g: 13.39 | ade_val_g: 12.57 | fde_train_g: 26.99 | fde_val_g: 23.91
e: 47 | loss: 1.38 | val_loss: 3.35 | ad

e: 85 | loss: 1.31 | val_loss: 2.83 | ade_train_l: 0.00 | ade_val_l: 0.00 | fde_train_l: 0.00 | fde_val_l: 0.00 | ade_train_g: 11.80 | ade_val_g: 11.81 | fde_train_g: 23.90 | fde_val_g: 22.04
e: 86 | loss: 1.31 | val_loss: 2.83 | ade_train_l: 0.00 | ade_val_l: 0.00 | fde_train_l: 0.00 | fde_val_l: 0.00 | ade_train_g: 12.01 | ade_val_g: 11.93 | fde_train_g: 23.90 | fde_val_g: 22.32
e: 87 | loss: 1.31 | val_loss: 3.34 | ade_train_l: 0.00 | ade_val_l: 0.00 | fde_train_l: 0.00 | fde_val_l: 0.00 | ade_train_g: 11.89 | ade_val_g: 12.51 | fde_train_g: 23.78 | fde_val_g: 22.95
e: 88 | loss: 1.29 | val_loss: 2.85 | ade_train_l: 0.00 | ade_val_l: 0.00 | fde_train_l: 0.00 | fde_val_l: 0.00 | ade_train_g: 11.60 | ade_val_g: 12.03 | fde_train_g: 23.16 | fde_val_g: 22.32
e: 89 | loss: 1.29 | val_loss: 3.39 | ade_train_l: 0.00 | ade_val_l: 0.00 | fde_train_l: 0.00 | fde_val_l: 0.00 | ade_train_g: 11.67 | ade_val_g: 12.90 | fde_train_g: 23.16 | fde_val_g: 24.16
e: 90 | loss: 1.29 | val_loss: 2.82 | ad

e: 128 | loss: 1.25 | val_loss: 2.91 | ade_train_l: 0.00 | ade_val_l: 0.00 | fde_train_l: 0.00 | fde_val_l: 0.00 | ade_train_g: 11.04 | ade_val_g: 12.72 | fde_train_g: 21.80 | fde_val_g: 24.76
e: 129 | loss: 1.25 | val_loss: 3.35 | ade_train_l: 0.00 | ade_val_l: 0.00 | fde_train_l: 0.00 | fde_val_l: 0.00 | ade_train_g: 11.00 | ade_val_g: 12.56 | fde_train_g: 21.82 | fde_val_g: 23.12
e: 130 | loss: 1.25 | val_loss: 3.85 | ade_train_l: 0.00 | ade_val_l: 0.00 | fde_train_l: 0.00 | fde_val_l: 0.00 | ade_train_g: 11.01 | ade_val_g: 12.88 | fde_train_g: 21.77 | fde_val_g: 22.99
e: 131 | loss: 1.25 | val_loss: 3.39 | ade_train_l: 0.00 | ade_val_l: 0.00 | fde_train_l: 0.00 | fde_val_l: 0.00 | ade_train_g: 11.02 | ade_val_g: 12.95 | fde_train_g: 21.85 | fde_val_g: 24.28
e: 132 | loss: 1.34 | val_loss: 3.40 | ade_train_l: 0.00 | ade_val_l: 0.00 | fde_train_l: 0.00 | fde_val_l: 0.00 | ade_train_g: 13.10 | ade_val_g: 13.14 | fde_train_g: 23.88 | fde_val_g: 24.54
e: 133 | loss: 1.36 | val_loss: 3.4

Epoch   171: reducing learning rate of group 0 to 1.5625e-03.
e: 170 | loss: 1.22 | val_loss: 2.88 | ade_train_l: 0.00 | ade_val_l: 0.00 | fde_train_l: 0.00 | fde_val_l: 0.00 | ade_train_g: 10.57 | ade_val_g: 12.67 | fde_train_g: 20.81 | fde_val_g: 24.36
e: 171 | loss: 1.22 | val_loss: 3.35 | ade_train_l: 0.00 | ade_val_l: 0.00 | fde_train_l: 0.00 | fde_val_l: 0.00 | ade_train_g: 10.58 | ade_val_g: 12.61 | fde_train_g: 20.72 | fde_val_g: 23.20
e: 172 | loss: 1.23 | val_loss: 2.87 | ade_train_l: 0.00 | ade_val_l: 0.00 | fde_train_l: 0.00 | fde_val_l: 0.00 | ade_train_g: 10.66 | ade_val_g: 12.45 | fde_train_g: 20.94 | fde_val_g: 23.72
e: 173 | loss: 1.22 | val_loss: 3.36 | ade_train_l: 0.00 | ade_val_l: 0.00 | fde_train_l: 0.00 | fde_val_l: 0.00 | ade_train_g: 10.62 | ade_val_g: 12.61 | fde_train_g: 20.83 | fde_val_g: 23.35
e: 174 | loss: 1.22 | val_loss: 2.86 | ade_train_l: 0.00 | ade_val_l: 0.00 | fde_train_l: 0.00 | fde_val_l: 0.00 | ade_train_g: 10.61 | ade_val_g: 12.24 | fde_train_g

e: 212 | loss: 1.30 | val_loss: 3.35 | ade_train_l: 0.00 | ade_val_l: 0.00 | fde_train_l: 0.00 | fde_val_l: 0.00 | ade_train_g: 12.58 | ade_val_g: 12.66 | fde_train_g: 22.60 | fde_val_g: 23.37
e: 213 | loss: 1.21 | val_loss: 3.37 | ade_train_l: 0.00 | ade_val_l: 0.00 | fde_train_l: 0.00 | fde_val_l: 0.00 | ade_train_g: 10.48 | ade_val_g: 12.78 | fde_train_g: 20.51 | fde_val_g: 23.58
e: 214 | loss: 1.22 | val_loss: 2.86 | ade_train_l: 0.00 | ade_val_l: 0.00 | fde_train_l: 0.00 | fde_val_l: 0.00 | ade_train_g: 10.51 | ade_val_g: 12.26 | fde_train_g: 20.67 | fde_val_g: 23.55
e: 215 | loss: 1.21 | val_loss: 2.86 | ade_train_l: 0.00 | ade_val_l: 0.00 | fde_train_l: 0.00 | fde_val_l: 0.00 | ade_train_g: 10.51 | ade_val_g: 12.27 | fde_train_g: 20.57 | fde_val_g: 23.42
e: 216 | loss: 1.21 | val_loss: 2.96 | ade_train_l: 0.00 | ade_val_l: 0.00 | fde_train_l: 0.00 | fde_val_l: 0.00 | ade_train_g: 10.46 | ade_val_g: 13.44 | fde_train_g: 20.48 | fde_val_g: 26.53
e: 217 | loss: 1.21 | val_loss: 2.8

e: 254 | loss: 1.22 | val_loss: 3.37 | ade_train_l: 0.00 | ade_val_l: 0.00 | fde_train_l: 0.00 | fde_val_l: 0.00 | ade_train_g: 10.59 | ade_val_g: 12.70 | fde_train_g: 20.73 | fde_val_g: 23.61
e: 255 | loss: 1.22 | val_loss: 2.85 | ade_train_l: 0.00 | ade_val_l: 0.00 | fde_train_l: 0.00 | fde_val_l: 0.00 | ade_train_g: 10.58 | ade_val_g: 12.17 | fde_train_g: 20.66 | fde_val_g: 23.25
e: 256 | loss: 1.22 | val_loss: 3.38 | ade_train_l: 0.00 | ade_val_l: 0.00 | fde_train_l: 0.00 | fde_val_l: 0.00 | ade_train_g: 10.51 | ade_val_g: 12.82 | fde_train_g: 20.57 | fde_val_g: 23.94
e: 257 | loss: 1.31 | val_loss: 2.86 | ade_train_l: 0.00 | ade_val_l: 0.00 | fde_train_l: 0.00 | fde_val_l: 0.00 | ade_train_g: 12.64 | ade_val_g: 12.37 | fde_train_g: 22.87 | fde_val_g: 23.62
e: 258 | loss: 1.21 | val_loss: 2.87 | ade_train_l: 0.00 | ade_val_l: 0.00 | fde_train_l: 0.00 | fde_val_l: 0.00 | ade_train_g: 10.45 | ade_val_g: 12.44 | fde_train_g: 20.45 | fde_val_g: 23.44
e: 259 | loss: 1.20 | val_loss: 3.3

e: 296 | loss: 1.21 | val_loss: 2.89 | ade_train_l: 0.00 | ade_val_l: 0.00 | fde_train_l: 0.00 | fde_val_l: 0.00 | ade_train_g: 10.43 | ade_val_g: 12.57 | fde_train_g: 20.42 | fde_val_g: 24.51
e: 297 | loss: 1.22 | val_loss: 2.86 | ade_train_l: 0.00 | ade_val_l: 0.00 | fde_train_l: 0.00 | fde_val_l: 0.00 | ade_train_g: 10.60 | ade_val_g: 12.39 | fde_train_g: 20.83 | fde_val_g: 23.80
Epoch   299: reducing learning rate of group 0 to 3.8147e-07.
e: 298 | loss: 1.22 | val_loss: 3.39 | ade_train_l: 0.00 | ade_val_l: 0.00 | fde_train_l: 0.00 | fde_val_l: 0.00 | ade_train_g: 10.63 | ade_val_g: 12.89 | fde_train_g: 20.89 | fde_val_g: 24.30
e: 299 | loss: 1.31 | val_loss: 3.40 | ade_train_l: 0.00 | ade_val_l: 0.00 | fde_train_l: 0.00 | fde_val_l: 0.00 | ade_train_g: 10.69 | ade_val_g: 13.17 | fde_train_g: 22.77 | fde_val_g: 24.53
Done !


In [94]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import torchvision
import torchvision.transforms as transforms

class myDataset_DE_depth(torch.utils.data.Dataset):
    def __init__(self, args, dtype):
        
        self.args = args
        self.dtype = dtype
        print("Loading",self.dtype)
        
        sequence_centric = pd.read_csv("sequences_openpifpaf_thres4_wimage_"+self.dtype+".csv")
#         sequences_openpifpaf_thres4_wconfsc_wimage_train
#         sequence_centric = pd.read_csv("sequences_16_overlap_4_thres4_"+self.dtype+".csv")

        df = sequence_centric.copy()      
        for v in list(df.columns.values):
            print(v+' loaded')
            try:
                df.loc[:,v] = df.loc[:, v].apply(lambda x: literal_eval(x))
            except:
                continue
        sequence_centric[df.columns] = df[df.columns]
        self.data = sequence_centric.copy().reset_index(drop=True)
        
        print('*'*30)
        
        self.obs={}
        self.true={}
        self.obs_act={}
        self.true_act={}
        
# #         
#             self.obs.append(obs_global)
#             self.true.append(true_global)
            
# #         
#         self.obs=np.array(self.obs)
#         self.true=np.array(self.true)
        

    def __len__(self):
        return len(self.data)
    
    
    def __getitem__(self, index):
#         print(index)
        
        outputs=[]
            
        if(index in self.obs):
#             print("here")
            obs=torch.tensor(self.obs[index],dtype=torch.double)
            true=torch.tensor(self.true[index],dtype=torch.double)
            obs_act=torch.tensor(self.obs_act[index],dtype=torch.double)
            true_act=torch.tensor(self.true_act[index],dtype=torch.double)
    
            outputs.append(obs)
            outputs.append(true)
            outputs.append(obs_act)
            outputs.append(true_act)
        
        else:
#             print("2nd")
            seq = self.data.iloc[index]
            outputs = []        
            obs_p=[]
            depth=[]
            obs_act=[]
            true_p=[]
            true_act=[]
            w=0
            h=0
            egomotion=np.loadtxt("./egomotion/"+seq.Pose_image[0].replace('/frames/','').replace('/','-').replace('.png','')+'_to_'+seq.Pose_image[-1].replace("/frames/","").replace('/','-').replace('.png','.txt'))
            for i in range(0,self.args.input,self.args.skip):
                image = seq.Pose_image[i].replace("/frames","./frames")
                depth_image=image.replace("frames","depth").replace(".png","_disp.png")
                im=cv2.imread(image,0)
                imd=cv2.imread(depth_image,0)
                h,w=im.shape
                hd,wd=imd.shape
                pose=seq.Pose[i][:2]    
#                 px=pose[0]/w
#                 py=pose[1]/h
                x=int(pose[0]*wd/w)
                y=int(pose[1]*hd/h)
#                 pose=[px,py]
                pose.append(imd[y,x])
                pose.extend(egomotion[i])
                obs_p.append(pose)
                obs_act.append([w,h])
                
                true=seq.Future_Pose[i][:2]
                true_p.append([true[0]/w, true[1]/h])
                true_act.append(true)

#             obs_global=obs_p
#             true = np.array([seq.Future_Pose[i] for i in range(0,self.args.output,self.args.skip)])

#             true_resh = true.reshape(true.shape[0],17,2)
#             true_global=true_resh[:,0]
#             true_global=[true_global[0]/w,true_global[1]/h]
#             print(obs_p)
#             print(obs_act)
            self.obs[index]=obs_p
            self.true[index]=true_p
            self.obs_act[index]=obs_act
            self.true_act[index]=true_act
            
#             print(self.obs[index])
#             print(self.true[index])
#             print(self.obs_act[index])
#             print(self.true_act[index])
            
            obs=torch.tensor(self.obs[index],dtype=torch.double)
            true=torch.tensor(self.true[index],dtype=torch.double)
            obs_act=torch.tensor(self.obs_act[index],dtype=torch.double)
            true_act=torch.tensor(self.true_act[index],dtype=torch.double)
    
            outputs.append(obs)
            outputs.append(true)
            outputs.append(obs_act)
            outputs.append(true_act)

       
        return tuple(outputs)    
    
    
def data_loader_DE_depth(args,data):
    dataset = myDataset_DE_depth(args,data)
    dataloader = torch.utils.data.DataLoader(
        dataset, batch_size=args.batch_size, shuffle=args.loader_shuffle,
        pin_memory=args.pin_memory)

    return dataloader

#Interpolation

In [95]:

class args():
    def __init__(self):
       
        self.dtype        = 'train'
        self.loader_workers = 1
        self.loader_shuffle = True
        self.pin_memory     = False
        self.device         = 'cuda'
        self.batch_size     = 50
        self.input  = 16
        self.output = 16
        self.stride = 16
        self.skip   = 1
        
args = args()

train_loader=data_loader_DE_depth(args,"train")
val_loader=data_loader_DE_depth(args,"val" )

Loading train
Pose loaded
Future_Pose loaded
Pose_image loaded
Future_image loaded
******************************
Loading val
Pose loaded
Future_Pose loaded
Pose_image loaded
Future_image loaded
******************************


In [201]:
torch.set_printoptions(precision=4,sci_mode =False)
for idx, (obs_pose_global, target_pose_global,obs_pose_act,target_pose_act) in enumerate(val_loader):
    obs_pose_global = obs_pose_global.to(device='cuda')
    target_pose_global = target_pose_global.to(device='cuda')
    (preds_g,) = net(pose=obs_pose_global)
    print(obs_pose_global[0])
#     print(target_pose_act[1][0])
#     print(preds_g[1][0])
    break

tensor([    70.0000,      1.0000,     -0.0002,      0.0000,     -0.0002,
             0.0002,      1.0000,     -0.0001,      0.0003,     -0.0000,
             0.0001,      1.0000,     -0.0010], dtype=torch.float64)
tensor([    73.0000,      1.0000,     -0.0007,      0.0001,     -0.0003,
             0.0007,      1.0000,     -0.0002,      0.0007,     -0.0001,
             0.0002,      1.0000,     -0.0017], dtype=torch.float64)
tensor([    75.0000,      1.0000,     -0.0003,     -0.0003,     -0.0000,
             0.0003,      1.0000,      0.0001,      0.0001,      0.0003,
            -0.0001,      1.0000,     -0.0045], dtype=torch.float64)
tensor([    76.0000,      1.0000,     -0.0005,     -0.0005,     -0.0001,
             0.0005,      1.0000,      0.0002,      0.0003,      0.0005,
            -0.0002,      1.0000,     -0.0090], dtype=torch.float64)
tensor([    22.0000,      1.0000,     -0.0005,      0.0000,     -0.0001,
             0.0005,      1.0000,     -0.0003,      0.0004,     -0.

ValueError: not enough values to unpack (expected 4, got 2)

In [173]:
class glob_LSTM_DE(nn.Module):    
    def __init__(self, args):
        '''
           input: 
           output: 
        '''
        super(glob_LSTM_DE, self).__init__()
         
        
        self.DE_encoder = nn.LSTM(input_size=2, hidden_size=100)
        self.DE_decoder = nn.LSTMCell(input_size=2, hidden_size=100)
#         self.vel_decoder = nn.LSTMCell(input_size=self.encoded_size, hidden_size=args.hidden_size)

        self.fc_DE  = nn.Linear(in_features=100, out_features=2)
#         self.fc2_DE  = nn.Linear(in_features=15, out_features=2)
        
#         self.hardtanh = nn.Hardtanh(min_val=-1*100,max_val=100)
        self.relu = nn.ReLU() 
#         self.softmax = nn.Softmax(dim=1)
        
        self.args = args
        
    def forward(self, pose=None):
        
        outputs = []        
        _, (hidden_dec,cell_dec) = self.DE_encoder(pose.permute(1,0,2))    
        hidden_dec=hidden_dec.squeeze(0)
        cell_dec=cell_dec.squeeze(0)
#         print(cell_dec.size())
#         print(pose.shape)
#         print(pose[1,-1])
        DEDec_inp = pose[:,-1,:]
       
        DE_outputs = torch.tensor([], device=self.args.device,dtype=torch.double)
        
#         print(DEDec_inp.shape)
        for i in range(self.args.output//self.args.skip):
#             print(hidden_dec[0].shape,hidden_dec[1].shape)
            (hidden_dec,cell_dec) = self.DE_decoder(DEDec_inp, (hidden_dec,cell_dec)) #decoder_output, 
            DE_output_t  = self.hardtanh(self.fc_DE(hidden_dec))
#             DE_output  = self.fc2_DE(DE_output_t)
#             print(DE_output_t.shape)
            DE_outputs = torch.cat((DE_outputs, DE_output_t.unsqueeze(1)), dim = 1)
            DEDec_inp  = DE_output_t.detach()
            
            
#         print(DE_outputs.shape).permute(1,0,2)
        outputs.append(DE_outputs)
            
        return tuple(outputs)

net=glob_LSTM_DE(args).to(args.device).double()

In [225]:
print('='*100)
print('Training ...')

train_p_scores=[]
val_p_scores=[]
alpha=1#0.4
l1e = nn.L1Loss()
train_s_scores = []
train_pose_scores=[]
val_pose_scores=[]
train_c_scores = []
val_s_scores   = []
val_c_scores   = []

optimizer = optim.Adam(net.parameters(), lr= 0.1)
# optimizer = optim.Adadelta(net.parameters(),lr= 0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=8, 
                                                 threshold = 1e-12, verbose=True)

for epoch in range(300):
    start = time.time()
    
    avg_epoch_train_p_loss   = 0
    avg_epoch_val_p_loss     = 0 
    ade_g  = 0
    fde_g  = 0
    ade_train_g  = 0
    fde_train_g  = 0
    counter = 0
    net.train()
    
    for idx, (obs_pose_global, target_pose_global,obs_s_global,target_s_global) in enumerate(train_loader):
        obs_pose_global = obs_pose_global.to(device='cuda')
        target_pose_global = target_pose_global.to(device='cuda')
        obs_s_global = obs_s_global.to(device='cuda')
        target_s_global = target_s_global.to(device='cuda')
        
        counter += 1        
        
        net.zero_grad()
#         print(obs_pose_global)
        (preds,) = net(pose=obs_s_global[:,:,:2])
        preds_g=speed2pos(preds,obs_pose_global)
        
        ade_train_g += float(ADE_c(preds_g, target_pose_global))
        fde_train_g += float(FDE_c(preds_g, target_pose_global))
      
        loss_g  = l1e(preds, target_s_global)
        
        loss=alpha*loss_g
    
        loss.backward()
        optimizer.step()
        
        avg_epoch_train_p_loss += float(loss)

    avg_epoch_train_p_loss /= counter
    ade_train_g  /= counter
    fde_train_g  /= counter   
    

    counter=0
    net.eval()
    for idx, (obs_pose_global, target_pose_global,obs_s_global,target_s_global) in enumerate(val_loader):
        obs_pose_global = obs_pose_global.to(device='cuda')
        target_pose_global = target_pose_global.to(device='cuda')
        obs_s_global = obs_s_global.to(device='cuda')
        target_s_global = target_s_global.to(device='cuda')
    
        counter += 1        

        with torch.no_grad():
            (preds,) = net(pose=obs_s_global[:,:,:2])
            preds_g=speed2pos(preds,obs_pose_global)
        
            ade_g += float(ADE_c(preds_g, target_pose_global))
            fde_g += float(FDE_c(preds_g, target_pose_global))

            loss_g  = l1e(preds, target_s_global)

            val_loss=alpha*loss_g

            avg_epoch_val_p_loss += float(val_loss)
      
    avg_epoch_val_p_loss /= counter
    val_p_scores.append(avg_epoch_val_p_loss)
    ade_g  /= counter
    fde_g  /= counter    
   
    
    
    scheduler.step(avg_epoch_train_p_loss)
    
    print('e:', epoch,'| loss: %.2f'%avg_epoch_train_p_loss,'| val_loss: %.2f'% avg_epoch_val_p_loss, '| ade_train_g: %.2f'% ade_train_g, '| ade_val_g: %.2f'% ade_g, '| fde_train_g: %.2f'% fde_train_g,'| fde_val_g: %.2f'% fde_g)


print('='*100) 
# print('Saving ...')
# torch.save(net.state_dict(), args.model_path)
print('Done !')

Training ...
e: 0 | loss: 413.96 | val_loss: 45.77 | ade_train_g: 1199.65 | ade_val_g: 211.18 | fde_train_g: 1597.73 | fde_val_g: 490.11
e: 1 | loss: 32.98 | val_loss: 15.91 | ade_train_g: 152.85 | ade_val_g: 74.08 | fde_train_g: 364.21 | fde_val_g: 135.10
e: 2 | loss: 14.46 | val_loss: 14.44 | ade_train_g: 88.14 | ade_val_g: 83.24 | fde_train_g: 176.68 | fde_val_g: 158.60
e: 3 | loss: 11.84 | val_loss: 12.59 | ade_train_g: 67.21 | ade_val_g: 61.37 | fde_train_g: 127.66 | fde_val_g: 114.23
e: 4 | loss: 9.95 | val_loss: 9.19 | ade_train_g: 64.11 | ade_val_g: 49.58 | fde_train_g: 120.64 | fde_val_g: 87.20
e: 5 | loss: 8.15 | val_loss: 9.49 | ade_train_g: 48.75 | ade_val_g: 43.86 | fde_train_g: 94.97 | fde_val_g: 77.51
e: 6 | loss: 7.10 | val_loss: 7.03 | ade_train_g: 41.84 | ade_val_g: 45.34 | fde_train_g: 79.40 | fde_val_g: 86.48
e: 7 | loss: 6.40 | val_loss: 7.64 | ade_train_g: 46.46 | ade_val_g: 47.79 | fde_train_g: 85.39 | fde_val_g: 91.23
e: 8 | loss: 6.26 | val_loss: 8.20 | ade_tra

e: 69 | loss: 1.70 | val_loss: 3.30 | ade_train_g: 16.31 | ade_val_g: 18.16 | fde_train_g: 32.93 | fde_val_g: 34.07
e: 70 | loss: 1.58 | val_loss: 3.79 | ade_train_g: 15.14 | ade_val_g: 17.38 | fde_train_g: 30.62 | fde_val_g: 32.48
e: 71 | loss: 1.58 | val_loss: 3.71 | ade_train_g: 14.99 | ade_val_g: 17.08 | fde_train_g: 30.12 | fde_val_g: 31.64
e: 72 | loss: 1.57 | val_loss: 3.71 | ade_train_g: 15.27 | ade_val_g: 17.26 | fde_train_g: 30.76 | fde_val_g: 31.71
e: 73 | loss: 1.54 | val_loss: 3.36 | ade_train_g: 14.69 | ade_val_g: 18.90 | fde_train_g: 29.80 | fde_val_g: 34.81
e: 74 | loss: 1.63 | val_loss: 3.36 | ade_train_g: 14.78 | ade_val_g: 18.53 | fde_train_g: 31.67 | fde_val_g: 34.26
e: 75 | loss: 1.56 | val_loss: 4.23 | ade_train_g: 15.27 | ade_val_g: 17.88 | fde_train_g: 30.80 | fde_val_g: 32.17
e: 76 | loss: 1.60 | val_loss: 3.42 | ade_train_g: 15.46 | ade_val_g: 19.25 | fde_train_g: 31.27 | fde_val_g: 36.76
e: 77 | loss: 1.56 | val_loss: 3.23 | ade_train_g: 14.91 | ade_val_g: 16

e: 138 | loss: 1.56 | val_loss: 3.66 | ade_train_g: 13.88 | ade_val_g: 15.63 | fde_train_g: 30.08 | fde_val_g: 29.27
e: 139 | loss: 1.46 | val_loss: 3.15 | ade_train_g: 13.50 | ade_val_g: 14.98 | fde_train_g: 27.52 | fde_val_g: 28.95
e: 140 | loss: 1.44 | val_loss: 3.64 | ade_train_g: 13.40 | ade_val_g: 15.27 | fde_train_g: 27.09 | fde_val_g: 28.69
e: 141 | loss: 1.44 | val_loss: 4.13 | ade_train_g: 13.34 | ade_val_g: 15.54 | fde_train_g: 26.95 | fde_val_g: 28.29
e: 142 | loss: 1.44 | val_loss: 3.79 | ade_train_g: 13.31 | ade_val_g: 16.43 | fde_train_g: 27.03 | fde_val_g: 31.52
Epoch   144: reducing learning rate of group 0 to 1.5625e-03.
e: 143 | loss: 1.45 | val_loss: 3.62 | ade_train_g: 13.59 | ade_val_g: 15.22 | fde_train_g: 27.36 | fde_val_g: 28.62
e: 144 | loss: 1.47 | val_loss: 3.66 | ade_train_g: 13.62 | ade_val_g: 15.83 | fde_train_g: 27.85 | fde_val_g: 29.91
e: 145 | loss: 1.43 | val_loss: 3.22 | ade_train_g: 13.23 | ade_val_g: 15.69 | fde_train_g: 26.90 | fde_val_g: 30.71
e:

e: 206 | loss: 1.41 | val_loss: 4.20 | ade_train_g: 12.90 | ade_val_g: 16.41 | fde_train_g: 25.98 | fde_val_g: 30.07
e: 207 | loss: 1.40 | val_loss: 3.18 | ade_train_g: 12.85 | ade_val_g: 15.24 | fde_train_g: 25.84 | fde_val_g: 29.27
e: 208 | loss: 1.42 | val_loss: 3.41 | ade_train_g: 12.96 | ade_val_g: 17.55 | fde_train_g: 26.14 | fde_val_g: 34.74
e: 209 | loss: 1.40 | val_loss: 4.79 | ade_train_g: 12.80 | ade_val_g: 17.51 | fde_train_g: 25.78 | fde_val_g: 32.03
e: 210 | loss: 1.40 | val_loss: 3.71 | ade_train_g: 12.80 | ade_val_g: 16.14 | fde_train_g: 25.80 | fde_val_g: 30.21
Epoch   212: reducing learning rate of group 0 to 4.8828e-05.
e: 211 | loss: 1.50 | val_loss: 3.18 | ade_train_g: 14.87 | ade_val_g: 15.52 | fde_train_g: 27.81 | fde_val_g: 29.35
e: 212 | loss: 1.41 | val_loss: 3.75 | ade_train_g: 12.92 | ade_val_g: 16.68 | fde_train_g: 26.09 | fde_val_g: 31.60
e: 213 | loss: 1.40 | val_loss: 3.25 | ade_train_g: 12.85 | ade_val_g: 16.05 | fde_train_g: 25.85 | fde_val_g: 31.48
e:

e: 273 | loss: 1.41 | val_loss: 4.67 | ade_train_g: 12.93 | ade_val_g: 16.45 | fde_train_g: 26.11 | fde_val_g: 29.48
e: 274 | loss: 1.40 | val_loss: 3.35 | ade_train_g: 12.78 | ade_val_g: 17.16 | fde_train_g: 25.70 | fde_val_g: 33.70
e: 275 | loss: 1.40 | val_loss: 3.68 | ade_train_g: 12.83 | ade_val_g: 15.67 | fde_train_g: 25.73 | fde_val_g: 29.40
e: 276 | loss: 1.41 | val_loss: 3.85 | ade_train_g: 12.84 | ade_val_g: 17.52 | fde_train_g: 25.78 | fde_val_g: 33.57
e: 277 | loss: 1.40 | val_loss: 3.68 | ade_train_g: 12.72 | ade_val_g: 15.80 | fde_train_g: 25.59 | fde_val_g: 29.74
e: 278 | loss: 1.49 | val_loss: 3.69 | ade_train_g: 14.80 | ade_val_g: 15.95 | fde_train_g: 27.54 | fde_val_g: 29.60
Epoch   280: reducing learning rate of group 0 to 3.8147e-07.
e: 279 | loss: 1.41 | val_loss: 3.28 | ade_train_g: 12.89 | ade_val_g: 16.48 | fde_train_g: 25.88 | fde_val_g: 32.03
e: 280 | loss: 1.42 | val_loss: 3.16 | ade_train_g: 12.99 | ade_val_g: 15.28 | fde_train_g: 26.28 | fde_val_g: 29.25
e:

In [84]:
for epoch in range(300):
    start = time.time()
    
    avg_epoch_train_p_loss   = 0
    avg_epoch_val_p_loss     = 0 
    ade_g  = 0
    fde_g  = 0
    ade_train_g  = 0
    fde_train_g  = 0
    counter = 0
    net.train()
    
    for idx, ( obs_pose,_,_,target_pose_act) in enumerate(train_loader):
            obs_pose_global = obs_pose_global.to(device='cuda')
            target_pose_global = target_pose_global.to(device='cuda')

ValueError: not enough values to unpack (expected 9, got 4)

In [136]:
dtype="train"
sequence_centric = pd.read_csv("sequences_openpifpaf_thres4_wimage_"+dtype+".csv")
df = sequence_centric.copy()      
for v in list(df.columns.values):
    print(v+' loaded')
    try:
        df.loc[:,v] = df.loc[:, v].apply(lambda x: literal_eval(x))
    except:
        continue
sequence_centric[df.columns] = df[df.columns]
data = sequence_centric.copy().reset_index(drop=True)
print('*'*30)

obs_pose=[]
true_pose=[]

outputs=[]

for index in range(len(data)):

    seq = data.iloc[index]
    obs_p=[]
    true_p=[]
    w=0
    h=0
    egomotion=np.loadtxt("./egomotion/"+seq.Pose_image[0].replace('/frames/','').replace('/','-').replace('.png','')+'_to_'+seq.Pose_image[-1].replace("/frames/","").replace('/','-').replace('.png','.txt'))
    for i in range(0,16,1):
        image = seq.Pose_image[i].replace("/frames","./frames")
        depth_image=image.replace("frames","depth").replace(".png","_disp.png")
        im=cv2.imread(image,0)
        imd=cv2.imread(depth_image,0)
        h,w=im.shape
        hd,wd=imd.shape
        pose=seq.Pose[i][:2]    
    #                 px=pose[0]/w
    #                 py=pose[1]/h
        x=int(pose[0]*wd/w)
        y=int(pose[1]*hd/h)
    #                 pose=[px,py]
        pose.append(imd[y,x])
        pose.extend(egomotion[i])
        obs_p.append(pose)
#         obs_act.append([w,h])

        true=seq.Future_Pose[i][:2]
#         true_p.append([true[0]/w, true[1]/h])
        true_p.append(true)
    
#     print(len(obs_p[0]))
#     print(len(true_p))
    
    obs_pose.append(obs_p)
    true_pose.append(true_p)
    print(index)
    
#     break

# print(len(obs_pose),len(true_pose))


Pose loaded
Future_Pose loaded
Pose_image loaded
Future_image loaded
******************************
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
25

In [137]:
data = {'Pose': obs_pose,
        'Future_Pose': true_pose,
        }

df_train = pd.DataFrame (data, columns = ['Pose','Future_Pose'])

df_train.to_csv("./glob_depth_train.csv",index=False)



In [ ]:
dtype="val"
sequence_centric = pd.read_csv("sequences_openpifpaf_thres4_wimage_"+dtype+".csv")
df = sequence_centric.copy()      
for v in list(df.columns.values):
    print(v+' loaded')
    try:
        df.loc[:,v] = df.loc[:, v].apply(lambda x: literal_eval(x))
    except:
        continue
sequence_centric[df.columns] = df[df.columns]
data = sequence_centric.copy().reset_index(drop=True)
print('*'*30)

obs_pose_val=[]
true_pose_val=[]

outputs=[]

for index in range(len(data)):

    seq = data.iloc[index]
    obs_p=[]
    true_p=[]
    w=0
    h=0
#     print(seq.Pose)
    egomotion=np.loadtxt("./egomotion/"+seq.Pose_image[0].replace('/frames/','').replace('/','-').replace('.png','')+'_to_'+seq.Pose_image[-1].replace("/frames/","").replace('/','-').replace('.png','.txt'))
    for i in range(0,16,1):
        image = seq.Pose_image[i].replace("/frames","./frames")
        depth_image=image.replace("frames","depth").replace(".png","_disp.png")
        im=cv2.imread(image,0)
        imd=cv2.imread(depth_image,0)
        h,w=im.shape
        hd,wd=imd.shape
        pose=seq.Pose[i][:2]    
    #                 px=pose[0]/w
    #                 py=pose[1]/h
        x=int(pose[0]*wd/w)
        y=int(pose[1]*hd/h)
    #                 pose=[px,py]
        pose.append(imd[y,x])
        pose.extend(egomotion[i])
        print(egomotion[i])
        obs_p.append(pose)
#         obs_act.append([w,h])

        true=seq.Future_Pose[i][:2]
#         true_p.append([true[0]/w, true[1]/h])
        true_p.append(true)
    
#     print(len(obs_p[0]))
#     print(len(true_p))
    
    obs_pose_val.append(obs_p)
    true_pose_val.append(true_p)
    print(index)
    
    break

# print(len(obs_pose),len(true_pose))


In [139]:
data_val = {'Pose': obs_pose_val,
        'Future_Pose': true_pose_val,
        }
df_val = pd.DataFrame (data_val, columns = ['Pose','Future_Pose'])
df_val.to_csv("./glob_depth_val.csv",index=False)

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import torchvision
import torchvision.transforms as transforms

class myDataset_DE_depth(torch.utils.data.Dataset):
    def __init__(self, args, dtype):
        
        self.args = args
        self.dtype = dtype
        print("Loading",self.dtype)
        
        sequence_centric = pd.read_csv("glob_depth_"+self.dtype+".csv")
#         sequences_openpifpaf_thres4_wconfsc_wimage_train
#         sequence_centric = pd.read_csv("sequences_16_overlap_4_thres4_"+self.dtype+".csv")

        df = sequence_centric.copy()      
        for v in list(df.columns.values):
            print(v+' loaded')
            try:
                df.loc[:,v] = df.loc[:, v].apply(lambda x: literal_eval(x))
            except:
                continue
        sequence_centric[df.columns] = df[df.columns]
        self.data = sequence_centric.copy().reset_index(drop=True)
        
        print('*'*30)
        
        self.obs=self.data.Pose
        self.true=self.data.Future_Pose
        

    def __len__(self):
        return len(self.data)
    
    
    def __getitem__(self, index):
#         print(index)
        
        outputs=[]
        
        obs=torch.tensor(self.obs[index],dtype=torch.double)
        true=torch.tensor(self.true[index],dtype=torch.double)
        
        outputs.append(obs[:,:2])
        outputs.append(true)
#         print(obs[1,2:])
        obs_speed = (obs[1:,:2] - obs[:-1,:2])
#         print(obs_speed)
        obs_speed=torch.cat((obs_speed,obs[1:,2:]),dim=1)
#         print(obs_speed[0,2:])
        
        outputs.append(obs_speed)
        
        true_speed = torch.cat(((true[0,:2]-obs[-1,:2]).unsqueeze(0), true[1:,:2]-true[:-1,:2]))
        outputs.append(true_speed)
#         outputs.append(obs)
#         outputs.append(true)
       
       
        return tuple(outputs)    
    
    
def data_loader_DE_depth(args,data):
    dataset = myDataset_DE_depth(args,data)
    dataloader = torch.utils.data.DataLoader(
        dataset, batch_size=args.batch_size, shuffle=args.loader_shuffle,
        pin_memory=args.pin_memory)

    return dataloader

#Interpolation

In [5]:

class args():
    def __init__(self):
       
        self.dtype        = 'train'
        self.loader_workers = 1
        self.loader_shuffle = True
        self.pin_memory     = False
        self.device         = 'cuda'
        self.batch_size     = 50
        self.input  = 16
        self.output = 16
        self.stride = 16
        self.skip   = 1
        
args = args()

train_loader=data_loader_DE_depth(args,"train")
val_loader=data_loader_DE_depth(args,"val" )

Loading train
Pose loaded
Future_Pose loaded
******************************
Loading val
Pose loaded
Future_Pose loaded
******************************


In [ ]:
torch.set_printoptions(precision=4,sci_mode =False)
for idx, (obs_pose_global, target_pose_global,obs_s_global,target_s_global) in enumerate(val_loader):
    obs_pose_global = obs_pose_global.to(device='cuda')
    target_pose_global = target_pose_global.to(device='cuda')
    obs_s_global = obs_s_global.to(device='cuda')
    target_s_global = target_s_global.to(device='cuda')
#     (preds_g,) = net(pose=obs_pose_global)
    print(obs_pose_global[0])
    print(target_pose_global[0])
    print(obs_s_global[0])
    print(target_s_global[0])
#     print(target_pose_act[1][0])
#     print(preds_g[1][0])
    break

In [7]:
class glob_LSTM_DE(nn.Module):    
    def __init__(self, args):
        '''
           input: 
           output: 
        '''
        super(glob_LSTM_DE, self).__init__()
         
        
        self.DE_encoder = nn.LSTM(input_size=15, hidden_size=100)
        self.DE_decoder = nn.LSTMCell(input_size=15, hidden_size=100)
#         self.vel_decoder = nn.LSTMCell(input_size=self.encoded_size, hidden_size=args.hidden_size)

        self.fc_DE  = nn.Linear(in_features=100, out_features=15)
        self.fc2_DE  = nn.Linear(in_features=15, out_features=2)
        
        self.hardtanh = nn.Hardtanh(min_val=-1*100,max_val=100)
        self.relu = nn.ReLU() 
#         self.softmax = nn.Softmax(dim=1)
        
        self.args = args
        
    def forward(self, pose=None):
        
        outputs = []        
        _, (hidden_dec,cell_dec) = self.DE_encoder(pose.permute(1,0,2))    
        hidden_dec=hidden_dec.squeeze(0)
        cell_dec=cell_dec.squeeze(0)
#         print(cell_dec.size())
#         print(pose.shape)
#         print(pose[1,-1])
        DEDec_inp = pose[:,-1,:]
       
        DE_outputs = torch.tensor([], device=self.args.device,dtype=torch.double)
        
#         print(DEDec_inp.shape)
        for i in range(self.args.output//self.args.skip):
#             print(hidden_dec[0].shape,hidden_dec[1].shape)
            (hidden_dec,cell_dec) = self.DE_decoder(DEDec_inp, (hidden_dec,cell_dec)) #decoder_output, 
            DE_output_t  = self.fc_DE(hidden_dec)
            DE_output  = self.fc2_DE(DE_output_t)
#             print(DE_output_t.shape)
            DE_outputs = torch.cat((DE_outputs, DE_output.unsqueeze(1)), dim = 1)
            DEDec_inp  = DE_output_t.detach()
            
            
#         print(DE_outputs.shape).permute(1,0,2)
        outputs.append(DE_outputs)
            
        return tuple(outputs)

net=glob_LSTM_DE(args).to(args.device).double()

In [11]:
print('='*100)
print('Training ...')

train_p_scores=[]
val_p_scores=[]
alpha=1#0.4
l1e = nn.L1Loss()
train_s_scores = []
train_pose_scores=[]
val_pose_scores=[]
train_c_scores = []
val_s_scores   = []
val_c_scores   = []

optimizer = optim.Adam(net.parameters(), lr= 0.1)
# optimizer = optim.Adadelta(net.parameters(),lr= 0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=8, 
                                                 threshold = 1e-12, verbose=True)

for epoch in range(300):
    start = time.time()
    
    avg_epoch_train_p_loss   = 0
    avg_epoch_val_p_loss     = 0 
    ade_g  = 0
    fde_g  = 0
    ade_train_g  = 0
    fde_train_g  = 0
    counter = 0
    net.train()
    
    for idx, (obs_pose_global, target_pose_global,obs_s_global,target_s_global) in enumerate(train_loader):
        obs_pose_global = obs_pose_global.to(device='cuda')
        target_pose_global = target_pose_global.to(device='cuda')
        obs_s_global = obs_s_global.to(device='cuda')
        target_s_global = target_s_global.to(device='cuda')
        
        counter += 1        
        
        net.zero_grad()
#         print(obs_pose_global)
        (preds,) = net(pose=obs_s_global)
        preds_g=speed2pos(preds,obs_pose_global)
        
        ade_train_g += float(ADE_c(preds_g, target_pose_global))
        fde_train_g += float(FDE_c(preds_g, target_pose_global))
      
        loss_g  = l1e(preds, target_s_global)
        
        loss=alpha*loss_g
    
        loss.backward()
        optimizer.step()
        
        avg_epoch_train_p_loss += float(loss)

    avg_epoch_train_p_loss /= counter
    ade_train_g  /= counter
    fde_train_g  /= counter   
    

    counter=0
    net.eval()
    for idx, (obs_pose_global, target_pose_global,obs_s_global,target_s_global) in enumerate(val_loader):
        obs_pose_global = obs_pose_global.to(device='cuda')
        target_pose_global = target_pose_global.to(device='cuda')
        obs_s_global = obs_s_global.to(device='cuda')
        target_s_global = target_s_global.to(device='cuda')
    
        counter += 1        

        with torch.no_grad():
            (preds,) = net(pose=obs_s_global)
            
            preds_g=speed2pos(preds,obs_pose_global)
        
            ade_g += float(ADE_c(preds_g, target_pose_global))
            fde_g += float(FDE_c(preds_g, target_pose_global))

            loss_g  = l1e(preds, target_s_global)

            val_loss=alpha*loss_g

            avg_epoch_val_p_loss += float(val_loss)
      
    avg_epoch_val_p_loss /= counter
    val_p_scores.append(avg_epoch_val_p_loss)
    ade_g  /= counter
    fde_g  /= counter    
   
    
    
    scheduler.step(avg_epoch_val_p_loss)
    
    print('e:', epoch,'| loss: %.2f'%avg_epoch_train_p_loss,'| val_loss: %.2f'% avg_epoch_val_p_loss, '| ade_train_g: %.2f'% ade_train_g, '| ade_val_g: %.2f'% ade_g, '| fde_train_g: %.2f'% fde_train_g,'| fde_val_g: %.2f'% fde_g)

print('='*100) 
# print('Saving ...')
# torch.save(net.state_dict(), args.model_path)
print('Done !')

Training ...
e: 0 | loss: 3.11 | val_loss: 4.91 | ade_train_g: 39.43 | ade_val_g: 38.73 | fde_train_g: 76.76 | fde_val_g: 72.26
e: 1 | loss: 2.84 | val_loss: 4.91 | ade_train_g: 37.63 | ade_val_g: 38.51 | fde_train_g: 72.41 | fde_val_g: 71.96
e: 2 | loss: 2.79 | val_loss: 4.42 | ade_train_g: 37.02 | ade_val_g: 38.95 | fde_train_g: 71.30 | fde_val_g: 73.45
e: 3 | loss: 2.79 | val_loss: 4.30 | ade_train_g: 36.95 | ade_val_g: 36.71 | fde_train_g: 71.41 | fde_val_g: 69.16
e: 4 | loss: 2.73 | val_loss: 4.03 | ade_train_g: 36.19 | ade_val_g: 32.82 | fde_train_g: 69.51 | fde_val_g: 60.27
e: 5 | loss: 2.60 | val_loss: 4.79 | ade_train_g: 33.06 | ade_val_g: 36.30 | fde_train_g: 63.94 | fde_val_g: 67.39
e: 6 | loss: 2.51 | val_loss: 4.74 | ade_train_g: 31.54 | ade_val_g: 33.07 | fde_train_g: 60.83 | fde_val_g: 62.64
e: 7 | loss: 2.60 | val_loss: 5.18 | ade_train_g: 32.33 | ade_val_g: 35.37 | fde_train_g: 63.14 | fde_val_g: 65.37
e: 8 | loss: 2.83 | val_loss: 4.39 | ade_train_g: 36.59 | ade_val_g

e: 70 | loss: 2.00 | val_loss: 3.69 | ade_train_g: 22.91 | ade_val_g: 25.87 | fde_train_g: 44.81 | fde_val_g: 49.41
e: 71 | loss: 2.29 | val_loss: 3.82 | ade_train_g: 26.94 | ade_val_g: 28.22 | fde_train_g: 53.97 | fde_val_g: 53.66
e: 72 | loss: 2.18 | val_loss: 4.05 | ade_train_g: 24.71 | ade_val_g: 24.39 | fde_train_g: 50.06 | fde_val_g: 45.55
e: 73 | loss: 1.97 | val_loss: 4.06 | ade_train_g: 22.82 | ade_val_g: 24.58 | fde_train_g: 44.76 | fde_val_g: 45.49
e: 74 | loss: 2.00 | val_loss: 3.58 | ade_train_g: 23.20 | ade_val_g: 24.20 | fde_train_g: 44.94 | fde_val_g: 45.74
e: 75 | loss: 1.96 | val_loss: 3.48 | ade_train_g: 22.53 | ade_val_g: 23.20 | fde_train_g: 44.18 | fde_val_g: 43.16
e: 76 | loss: 1.94 | val_loss: 3.51 | ade_train_g: 22.03 | ade_val_g: 22.60 | fde_train_g: 43.63 | fde_val_g: 43.36
Epoch    78: reducing learning rate of group 0 to 2.5000e-02.
e: 77 | loss: 2.02 | val_loss: 3.55 | ade_train_g: 23.31 | ade_val_g: 23.30 | fde_train_g: 45.91 | fde_val_g: 44.14
e: 78 | lo

e: 139 | loss: 1.58 | val_loss: 3.59 | ade_train_g: 16.24 | ade_val_g: 16.49 | fde_train_g: 31.72 | fde_val_g: 29.95
e: 140 | loss: 1.58 | val_loss: 3.14 | ade_train_g: 16.18 | ade_val_g: 16.63 | fde_train_g: 31.55 | fde_val_g: 31.02
e: 141 | loss: 1.57 | val_loss: 3.22 | ade_train_g: 15.94 | ade_val_g: 17.79 | fde_train_g: 31.16 | fde_val_g: 33.76
e: 142 | loss: 1.60 | val_loss: 3.11 | ade_train_g: 16.34 | ade_val_g: 16.44 | fde_train_g: 31.99 | fde_val_g: 30.63
e: 143 | loss: 1.58 | val_loss: 3.17 | ade_train_g: 16.26 | ade_val_g: 17.15 | fde_train_g: 31.73 | fde_val_g: 32.26
Epoch   145: reducing learning rate of group 0 to 1.5625e-03.
e: 144 | loss: 1.67 | val_loss: 3.16 | ade_train_g: 16.23 | ade_val_g: 16.86 | fde_train_g: 33.54 | fde_val_g: 31.75
e: 145 | loss: 1.57 | val_loss: 3.11 | ade_train_g: 15.96 | ade_val_g: 16.15 | fde_train_g: 31.23 | fde_val_g: 30.21
e: 146 | loss: 1.56 | val_loss: 3.15 | ade_train_g: 15.87 | ade_val_g: 16.78 | fde_train_g: 30.90 | fde_val_g: 31.64
e:

e: 206 | loss: 1.55 | val_loss: 3.10 | ade_train_g: 15.73 | ade_val_g: 15.97 | fde_train_g: 30.72 | fde_val_g: 30.01
e: 207 | loss: 1.56 | val_loss: 3.08 | ade_train_g: 15.75 | ade_val_g: 16.03 | fde_train_g: 30.86 | fde_val_g: 29.92
e: 208 | loss: 1.55 | val_loss: 3.62 | ade_train_g: 15.62 | ade_val_g: 16.62 | fde_train_g: 30.45 | fde_val_g: 30.76
e: 209 | loss: 1.56 | val_loss: 3.14 | ade_train_g: 15.76 | ade_val_g: 16.72 | fde_train_g: 30.83 | fde_val_g: 31.60
Epoch   211: reducing learning rate of group 0 to 1.2207e-05.
e: 210 | loss: 1.55 | val_loss: 3.09 | ade_train_g: 15.57 | ade_val_g: 15.97 | fde_train_g: 30.46 | fde_val_g: 29.81
e: 211 | loss: 1.54 | val_loss: 3.14 | ade_train_g: 15.54 | ade_val_g: 16.40 | fde_train_g: 30.34 | fde_val_g: 31.00
e: 212 | loss: 1.56 | val_loss: 3.07 | ade_train_g: 15.82 | ade_val_g: 15.72 | fde_train_g: 30.96 | fde_val_g: 29.37
e: 213 | loss: 1.57 | val_loss: 3.56 | ade_train_g: 15.80 | ade_val_g: 16.09 | fde_train_g: 31.02 | fde_val_g: 29.30
e:

Epoch   274: reducing learning rate of group 0 to 9.5367e-08.
e: 273 | loss: 1.54 | val_loss: 3.09 | ade_train_g: 15.55 | ade_val_g: 16.19 | fde_train_g: 30.40 | fde_val_g: 30.25
e: 274 | loss: 1.55 | val_loss: 3.11 | ade_train_g: 15.65 | ade_val_g: 16.26 | fde_train_g: 30.63 | fde_val_g: 30.63
e: 275 | loss: 1.55 | val_loss: 3.10 | ade_train_g: 15.68 | ade_val_g: 16.17 | fde_train_g: 30.70 | fde_val_g: 30.11
e: 276 | loss: 1.57 | val_loss: 3.13 | ade_train_g: 15.91 | ade_val_g: 16.55 | fde_train_g: 31.12 | fde_val_g: 30.90
e: 277 | loss: 1.56 | val_loss: 3.08 | ade_train_g: 15.78 | ade_val_g: 15.76 | fde_train_g: 30.88 | fde_val_g: 29.45
e: 278 | loss: 1.55 | val_loss: 3.14 | ade_train_g: 15.67 | ade_val_g: 16.52 | fde_train_g: 30.65 | fde_val_g: 31.50
e: 279 | loss: 1.55 | val_loss: 3.09 | ade_train_g: 15.57 | ade_val_g: 15.91 | fde_train_g: 30.45 | fde_val_g: 29.94
e: 280 | loss: 1.58 | val_loss: 3.11 | ade_train_g: 15.98 | ade_val_g: 16.13 | fde_train_g: 31.40 | fde_val_g: 30.34
e:

In [38]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        # encoder
#         self.drop =  nn.Dropout(p=0.3)
        self.l1= nn.Linear(in_features=15*15, out_features=300)
        self.l2 = nn.Linear(in_features=300, out_features=500)
        self.l3 = nn.Linear(in_features=500, out_features=16*2)
#         self.hardtanh = nn.Hardtanh(min_val=-100,max_val=100)
        self.relu = nn.ReLU() 

        
    def forward(self, x):
#         x = self.drop(x)
        x = self.l1(x)
        x = self.l2(x)
        x = self.l3(x)
        return x
net = MLP().double().to('cuda:0')
print(net)

MLP(
  (l1): Linear(in_features=225, out_features=300, bias=True)
  (l2): Linear(in_features=300, out_features=500, bias=True)
  (l3): Linear(in_features=500, out_features=32, bias=True)
  (relu): ReLU()
)


In [39]:
print('='*100)
print('Training ...')

l1e = nn.L1Loss()

optimizer = optim.Adam(net.parameters(), lr= 0.1)
# optimizer = optim.Adadelta(net.parameters(),lr= 0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=8, 
                                                 threshold = 1e-12, verbose=True)

for epoch in range(300):
    start = time.time()
    
    avg_epoch_train_p_loss   = 0
    avg_epoch_val_p_loss     = 0 
    ade_g  = 0
    fde_g  = 0
    ade_train_g  = 0
    fde_train_g  = 0
    counter = 0
    net.train()
    
    for idx, (obs_pose_global, target_pose_global,obs_s_global,target_s_global) in enumerate(train_loader):
        obs_pose_global = obs_pose_global.to(device='cuda')
        target_pose_global = target_pose_global.to(device='cuda')
        obs_s_global = obs_s_global.reshape(-1,15*15).to(device='cuda')
        target_s_global = target_s_global.to(device='cuda')
        
        counter += 1        
        
        net.zero_grad()
#         print(obs_pose_global.shape)
        preds = net(obs_s_global)
        preds_c=preds.clone().detach().reshape(-1,16,2)
#         print(preds_c.shape,obs_pose_global.shape)
        preds_g=speed2pos(preds_c,obs_pose_global)
        
        ade_train_g += float(ADE_c(preds_g, target_pose_global))
        fde_train_g += float(FDE_c(preds_g, target_pose_global))
      
        loss  = l1e(preds, target_s_global.reshape(-1,16*2))
        loss.backward()
        optimizer.step()
        
        avg_epoch_train_p_loss += float(loss)

    avg_epoch_train_p_loss /= counter
    ade_train_g  /= counter
    fde_train_g  /= counter   
    

    counter=0
    net.eval()
    for idx, (obs_pose_global, target_pose_global,obs_s_global,target_s_global) in enumerate(val_loader):
        obs_pose_global = obs_pose_global.to(device='cuda')
        target_pose_global = target_pose_global.to(device='cuda')
        obs_s_global = obs_s_global.reshape(-1,15*15).to(device='cuda')
        target_s_global = target_s_global.to(device='cuda')
    
        counter += 1        

        with torch.no_grad():
            preds = net(obs_s_global)
            preds_c=preds.clone().detach().reshape(-1,16,2)
            
            preds_g=speed2pos(preds_c,obs_pose_global)

            ade_g += float(ADE_c(preds_g, target_pose_global))
            fde_g += float(FDE_c(preds_g, target_pose_global))

            val_loss  = l1e(preds, target_s_global.reshape(-1,16*2))
            
            avg_epoch_val_p_loss += float(val_loss)
      
    avg_epoch_val_p_loss /= counter
    ade_g  /= counter
    fde_g  /= counter    
   
    
    
    scheduler.step(avg_epoch_train_p_loss)
    
    print('e:', epoch,'| loss: %.2f'%avg_epoch_train_p_loss,'| val_loss: %.2f'% avg_epoch_val_p_loss, '| ade_train_g: %.2f'% ade_train_g, '| ade_val_g: %.2f'% ade_g, '| fde_train_g: %.2f'% fde_train_g,'| fde_val_g: %.2f'% fde_g)

print('='*100) 
# print('Saving ...')
# torch.save(net.state_dict(), args.model_path)
print('Done !')

Training ...
e: 0 | loss: 71722.16 | val_loss: 36601.91 | ade_train_g: 1179.58 | ade_val_g: 1286.54 | fde_train_g: 1298.53 | fde_val_g: 1240.90
e: 1 | loss: 40069.82 | val_loss: 29401.06 | ade_train_g: 1325.71 | ade_val_g: 1313.54 | fde_train_g: 1228.29 | fde_val_g: 1621.92
e: 2 | loss: 14716.96 | val_loss: 14183.49 | ade_train_g: 1346.72 | ade_val_g: 1370.28 | fde_train_g: 1499.46 | fde_val_g: 1668.50
e: 3 | loss: 5866.26 | val_loss: 3914.52 | ade_train_g: 1198.80 | ade_val_g: 1422.24 | fde_train_g: 1303.69 | fde_val_g: 1548.92
e: 4 | loss: 2772.23 | val_loss: 1935.97 | ade_train_g: 1357.47 | ade_val_g: 1188.96 | fde_train_g: 1526.01 | fde_val_g: 1286.61
e: 5 | loss: 1008.57 | val_loss: 501.34 | ade_train_g: 1073.39 | ade_val_g: 734.16 | fde_train_g: 1134.06 | fde_val_g: 683.03
e: 6 | loss: 313.25 | val_loss: 271.74 | ade_train_g: 764.79 | ade_val_g: 794.18 | fde_train_g: 907.36 | fde_val_g: 985.15
e: 7 | loss: 133.70 | val_loss: 120.97 | ade_train_g: 469.92 | ade_val_g: 221.42 | fde_

e: 70 | loss: 1.63 | val_loss: 3.97 | ade_train_g: 10.78 | ade_val_g: 17.23 | fde_train_g: 24.26 | fde_val_g: 32.05
e: 71 | loss: 1.90 | val_loss: 4.32 | ade_train_g: 11.88 | ade_val_g: 19.45 | fde_train_g: 26.02 | fde_val_g: 31.80
e: 72 | loss: 1.72 | val_loss: 3.93 | ade_train_g: 11.58 | ade_val_g: 16.45 | fde_train_g: 26.36 | fde_val_g: 28.65
e: 73 | loss: 1.73 | val_loss: 3.90 | ade_train_g: 13.14 | ade_val_g: 17.92 | fde_train_g: 26.78 | fde_val_g: 31.51
e: 74 | loss: 1.79 | val_loss: 4.15 | ade_train_g: 11.62 | ade_val_g: 16.60 | fde_train_g: 27.46 | fde_val_g: 29.23
e: 75 | loss: 1.87 | val_loss: 4.76 | ade_train_g: 12.08 | ade_val_g: 16.42 | fde_train_g: 26.17 | fde_val_g: 29.41
e: 76 | loss: 2.00 | val_loss: 3.94 | ade_train_g: 11.56 | ade_val_g: 16.41 | fde_train_g: 26.24 | fde_val_g: 28.67
e: 77 | loss: 1.87 | val_loss: 6.03 | ade_train_g: 11.57 | ade_val_g: 18.65 | fde_train_g: 26.34 | fde_val_g: 31.18
e: 78 | loss: 2.86 | val_loss: 4.94 | ade_train_g: 12.59 | ade_val_g: 17

e: 140 | loss: 1.27 | val_loss: 3.47 | ade_train_g: 10.39 | ade_val_g: 15.94 | fde_train_g: 23.54 | fde_val_g: 27.62
e: 141 | loss: 1.22 | val_loss: 3.52 | ade_train_g: 9.98 | ade_val_g: 15.98 | fde_train_g: 22.60 | fde_val_g: 28.60
e: 142 | loss: 1.20 | val_loss: 4.01 | ade_train_g: 9.76 | ade_val_g: 15.65 | fde_train_g: 22.34 | fde_val_g: 26.95
e: 143 | loss: 1.33 | val_loss: 4.09 | ade_train_g: 10.54 | ade_val_g: 16.59 | fde_train_g: 23.25 | fde_val_g: 29.06
e: 144 | loss: 1.28 | val_loss: 4.07 | ade_train_g: 9.95 | ade_val_g: 16.50 | fde_train_g: 23.12 | fde_val_g: 28.26
e: 145 | loss: 1.25 | val_loss: 3.54 | ade_train_g: 9.80 | ade_val_g: 16.25 | fde_train_g: 22.64 | fde_val_g: 29.13
e: 146 | loss: 1.23 | val_loss: 4.10 | ade_train_g: 9.90 | ade_val_g: 16.96 | fde_train_g: 22.70 | fde_val_g: 29.66
e: 147 | loss: 1.25 | val_loss: 3.49 | ade_train_g: 10.07 | ade_val_g: 15.81 | fde_train_g: 23.22 | fde_val_g: 27.71
e: 148 | loss: 1.22 | val_loss: 4.04 | ade_train_g: 9.86 | ade_val_g:

e: 209 | loss: 1.18 | val_loss: 4.04 | ade_train_g: 9.76 | ade_val_g: 16.57 | fde_train_g: 22.54 | fde_val_g: 28.63
e: 210 | loss: 1.18 | val_loss: 5.14 | ade_train_g: 9.72 | ade_val_g: 20.65 | fde_train_g: 22.40 | fde_val_g: 33.20
e: 211 | loss: 1.18 | val_loss: 4.53 | ade_train_g: 9.62 | ade_val_g: 16.88 | fde_train_g: 22.16 | fde_val_g: 28.93
e: 212 | loss: 1.19 | val_loss: 4.56 | ade_train_g: 9.78 | ade_val_g: 16.74 | fde_train_g: 22.57 | fde_val_g: 28.54
e: 213 | loss: 1.18 | val_loss: 4.06 | ade_train_g: 9.71 | ade_val_g: 16.83 | fde_train_g: 22.42 | fde_val_g: 29.15
Epoch   215: reducing learning rate of group 0 to 9.7656e-05.
e: 214 | loss: 1.20 | val_loss: 4.01 | ade_train_g: 9.96 | ade_val_g: 16.33 | fde_train_g: 22.93 | fde_val_g: 28.17
e: 215 | loss: 1.17 | val_loss: 4.19 | ade_train_g: 9.71 | ade_val_g: 20.44 | fde_train_g: 22.28 | fde_val_g: 34.58
e: 216 | loss: 1.17 | val_loss: 4.65 | ade_train_g: 9.61 | ade_val_g: 20.15 | fde_train_g: 22.21 | fde_val_g: 32.64
e: 217 | l

e: 277 | loss: 1.17 | val_loss: 3.52 | ade_train_g: 9.66 | ade_val_g: 16.08 | fde_train_g: 22.35 | fde_val_g: 28.43
e: 278 | loss: 1.17 | val_loss: 4.16 | ade_train_g: 9.72 | ade_val_g: 19.90 | fde_train_g: 22.43 | fde_val_g: 33.25
e: 279 | loss: 1.17 | val_loss: 3.51 | ade_train_g: 9.62 | ade_val_g: 15.85 | fde_train_g: 22.26 | fde_val_g: 28.00
Epoch   281: reducing learning rate of group 0 to 3.0518e-06.
e: 280 | loss: 1.18 | val_loss: 4.01 | ade_train_g: 9.74 | ade_val_g: 16.21 | fde_train_g: 22.50 | fde_val_g: 27.99
e: 281 | loss: 1.18 | val_loss: 3.52 | ade_train_g: 9.77 | ade_val_g: 16.15 | fde_train_g: 22.62 | fde_val_g: 28.36
e: 282 | loss: 1.17 | val_loss: 4.05 | ade_train_g: 9.68 | ade_val_g: 16.62 | fde_train_g: 22.40 | fde_val_g: 28.93
e: 283 | loss: 1.18 | val_loss: 4.52 | ade_train_g: 9.74 | ade_val_g: 16.75 | fde_train_g: 22.43 | fde_val_g: 28.19
e: 284 | loss: 1.16 | val_loss: 3.54 | ade_train_g: 9.59 | ade_val_g: 16.33 | fde_train_g: 22.15 | fde_val_g: 29.00
e: 285 | l

In [110]:
import json

with open("./Posetrack/3dpw_train_in.json", "r") as read_file:
    data = json.load(read_file)


In [111]:
posesin=[]
# posesout=[]
for i in data:
#     print(len(i))
    for j in i:
        posesin.append(j)
# posesin=np.array(posesin)
print(len(posesin))

442


In [112]:
with open("./Posetrack/3dpw_train_out.json", "r") as read_file:
    data = json.load(read_file)
      
posesout=[]
for i in data:
    for j in i:
        posesout.append(j)
# posesout=np.array(posesout)
# print(posesout.shape)

In [105]:
with open("./Posetrack/3dpw_valid_masks_in.json", "r") as read_file:
    data = json.load(read_file)
      
maskin=[]
for i in data:
    for j in i:
        maskin.append(j)
mm=np.array(maskin)
print(mm.shape)

FileNotFoundError: [Errno 2] No such file or directory: './Posetrack/3dpw_valid_masks_in.json'

In [102]:
with open("./Posetrack/3dpw_train_masks_out.json", "r") as read_file:
    data = json.load(read_file)
      
maskout=[]
for i in data:
    for j in i:
        maskout.append(j)
# posesout=np.array(posesout)
# print(maskout.shape)

In [ ]:
with open("./Posetrack/posetrack_valid_masks_out.json", "r") as read_file:
    data = json.load(read_file)
      
maskout=[]
for i in data:
    for j in i:
        maskout.append(j)
# posesout=np.array(posesout)
# print(maskout.shape)

In [113]:
data_val = {'Pose': posesin,
            'Future_Pose': posesout,
#             'Mask': maskin,
#             'Future_Mask': maskout
        }
df = pd.DataFrame (data_val, columns = ['Pose','Future_Pose'])#,'Mask','Future_Mask'
df.to_csv("./3dpw_train.csv",index=False)

605
